In [343]:
from datetime import datetime
from threading import Thread
from time import sleep


from pathlib import Path

import cx_Oracle
import csv
connection = cx_Oracle.connect("RPA_ROBOT", "Qaz123123*", "REPS")

cursor = connection.cursor()
print(connection)
query = """        
select lntw.code_wares, ud.abr_unit, w.name_wares, NVL(wp.property_value, 1), name_node
from SPR.NODE_ALTERNATIVE_TREES         nat,
    SPR.LINK_NODE_TO_WARES             lntw,
    spr.alternative_trees              atr,
    spr.wares                          w
inner join spr.addition_unit au on (au.code_wares = w.code_wares and au.default_unit = 'Y')
inner join spr.unit_dimension ud on (ud.code_unit = au.code_unit)
left join spr.wares_property wp on (wp.code_wares = w.code_wares and wp.code_property = 3604)
where nat.code_node = lntw.code_node
and atr.code_tree = nat.code_tree
and atr.mapping_of_link = '11'
and nat.code_tree in (68323,68325,68326,68327,68328,68329,68330,68331,68332,68333,68354,68355,68356,68484,68357,68358,68359,68360,68361,68362,68363,68364,68365,68366,68367,68368,68394,68395,68401,68402,68403,68404,68407)
and w.code_wares = lntw.code_wares
"""
cursor.execute(query)

with open(('single.csv'), 'w+', newline='', encoding='windows-1251') as file:
    writer = csv.writer(file, delimiter=';')
    for i in cursor:
        writer.writerows([i])


<cx_Oracle.Connection to RPA_ROBOT@REPS>


In [344]:
import pandas as pd

df = pd.read_csv('single.csv', delimiter=';', header=None, encoding='windows-1251')
df

,0,1,2,3,4
0,15799,кг.,ГОЛЕНЬ ЦБ АЛЕЛЬ КГ,1.00,"Kуры(включая цыплят), свежий и охл., части ту..."
1,30694,кг.,КОСТИ КУРИНЫЕ/ОКОРОЧКА,1.00,"Kуры(включая цыплят), свежий и охл., части ту..."
2,34421,кг.,ГОЛЕНЬ Ц/Б С/ПР КГ,1.00,"Kуры(включая цыплят), свежий и охл., части ту..."
3,34423,кг.,МЯКОТЬ Ц/Б С/ПР КГ,1.00,"Kуры(включая цыплят), свежий и охл., части ту..."
4,34426,кг.,БЕДРО Ц/Б С/ПР КГ,1.00,"Kуры(включая цыплят), свежий и охл., части ту..."
...,...,...,...,...,...
4434,256628,кг.,ВАРЕНИКИ С КУРАГОЙ ОТВАРНЫЕ,1.00,Вареники
4435,256629,кг.,ВАРЕНИКИ С ЧЕРНОСЛИВОМ ОТВАРНЫЕ,1.00,Вареники
4436,264464,шт.,ВАРЕНИНИКИ С КАРТОФЕЛЕМ ОТВАРНЫЕ 200Г,0.30,Вареники
4437,271643,шт.,ДОСТАВКА ВАРЕНИКИ С КАРТОФЕЛЕМ ОТВАРНЫЕ 250Г,0.25,Вареники


In [359]:
import psycopg2
import csv
import pandas as pd

conn = psycopg2.connect(dbname='adb', host='172.16.10.22', port='5432',
                        user='rpa_robot', password='Qaz123123+')

cur = conn.cursor(name='1583_first_part')

start_date = '2023-01-01'
end_date = '2023-03-31'
store_id = '985'

df = pd.read_csv('single.csv', delimiter=';', header=None, encoding='windows-1251')

query = f"""
    select source_product_id, round(avg(avg_price), 2), sum(quantity) FROM "dwh_data"."qs_fact_sales" q where source_product_id in ('15799', '30694', '34421', '34423', '34426', '34427', '34431', '34432', '34433', '34434', '34435', '34439', '34441', '34443', '35207', '48284', '48706', '71322', '73600', '106563', '106564', '106565', '106566', '106567', '106568', '106572', '106699', '109309', '109310', '109311', '109312', '109314', '109317', '111062', '122917', '122919', '122920', '122924', '123321', '123322', '123325', '123326', '123327', '130737', '130739', '130740', '130741', '130742', '130743', '147761', '147762', '147763', '147764', '147765', '147766', '147767', '147768', '151021', '175464', '175470', '175473', '175475', '175480', '176650', '190811', '190812', '193472', '193473', '193474', '193475', '193476', '193478', '193479', '193480', '193481', '193484', '193487', '193488', '193489', '193490', '193492', '193493', '198679', '198680', '198681', '198682', '198683', '198684', '198688', '203379', '205522', '206026', '206027', '206028', '206029', '206030', '206031', '206038', '206255', '206256', '206350', '206351', '206352', '210968', '210970', '213129', '213133', '213134', '213135', '213136', '213137', '213138', '219749', '219750', '219751', '219752', '219753', '219756', '226766', '226767', '226768', '226769', '226770', '226771', '227317', '227318', '227319', '227320', '227321', '233399', '233401', '234646', '234647', '234653', '234666', '234689', '234690', '234693', '240314', '240316', '240317', '242447', '242449', '242451', '245120', '247451', '247482', '247483', '250527', '250528', '250529', '250530', '250531', '250532', '251430', '251431', '252241', '252941', '257337', '257338', '263169', '267398', '267538', '272212', '272213', '272214', '272215', '272216', '272217', '275333', '290672', '295432', '295436', '299183', '299185', '9515', '9519', '9524', '9525', '9528', '9529', '9530', '9543', '9551', '48047', '48048', '48050', '48051', '48491', '78059', '142555', '145640', '170319', '175462', '193458', '193459', '193460', '193461', '193462', '193463', '193464', '193465', '193466', '210969', '241080', '256601', '256602', '258397', '258398', '258399', '258400', '258800', '258801', '258802', '258803', '258872', '259019', '259020', '259021', '259023', '262733', '263014', '263015', '263016', '263017', '263018', '263019', '263020', '263021', '263022', '263023', '263024', '263025', '263026', '263027', '263028', '263029', '263030', '263031', '263032', '263033', '263034', '263035', '263036', '263037', '263038', '263039', '263041', '263115', '264584', '266224', '272207', '272208', '275894', '278473', '287210', '298501', '16508', '19408', '47295', '58632', '58634', '58635', '58657', '58670', '58671', '58693', '58718', '58732', '58758', '58872', '58914', '90158', '96938', '117945', '136163', '150350', '206864', '210342', '214115', '230310', '230311', '235783', '235925', '257315', '264452', '264739', '266744', '267607', '270457', '270458', '270473', '271625', '271673', '279741', '7039', '8472', '8851', '9914', '9916', '9917', '10077', '10079', '10080', '10081', '10105', '10731', '10732', '14852', '14853', '14935', '15723', '15732', '15832', '16912', '18087', '18088', '19035', '27323', '30968', '30969', '30971', '30980', '31001', '34589', '34595', '41232', '41233', '41235', '50733', '53093', '53906', '58583', '58593', '58626', '58628', '58672', '58695', '58696', '58727', '58743', '58777', '58783', '58828', '58829', '58833', '58842', '58889', '58891', '58905', '58918', '58919', '58923', '58928', '58931', '58947', '68704', '69282', '70223', '79047', '96939', '116694', '116696', '118566', '124383', '138299', '139092', '139094', '150373', '152155', '152165', '152167', '152180', '159886', '167149', '174784', '186008', '190250', '191488', '191490', '205661', '206865', '210186', '210259', '210344', '210386', '212419', '233594', '234055', '237087', '251748', '257317', '264360', '264442', '264443', '264444', '264446', '264447', '264448', '264451', '264453', '264454', '264455', '264456', '264457', '264458', '264459', '264460', '264463', '264466', '264721', '264722', '264723', '264724', '264736', '264738', '264741', '264742', '264745', '264746', '266661', '266671', '266676', '266677', '266679', '266686', '266690', '266691', '266693', '266695', '266696', '266752', '266753', '267595', '267597', '267812', '268042', '268044', '268243', '268529', '268949', '268988', '269086', '269571', '269621', '269635', '271604', '271605', '271606', '271633', '271640', '271650', '271654', '271667', '273360', '274189', '274406', '274835', '278055', '278646', '280538', '280540', '280542', '284822', '288141', '301184', '301220', '307569', '309161', '309293', '309295', '309301', '313746', '314032', '314033', '6244', '9836', '10084', '10087', '10090', '10094', '10104', '10727', '10728', '10729', '15790', '15791', '15795', '16501', '18089', '18090', '18373', '27332', '30937', '30957', '30976', '30977', '30979', '30999', '32800', '33435', '33440', '34425', '34451', '34455', '34458', '34459', '34545', '34547', '34548', '34596', '46379', '47116', '47124', '47146', '53095', '57665', '58033', '58036', '58039', '58041', '58096', '58097', '58098', '58099', '58100', '58102', '58585', '58586', '58625', '58753', '58775', '58776', '58846', '58874', '58894', '58895', '58925', '58948', '70224', '71312', '71315', '71316', '71321', '71328', '73615', '74406', '75937', '85055', '86247', '101096', '101097', '116691', '116692', '116693', '116794', '119370', '120009', '137873', '137875', '139093', '149695', '150347', '151680', '151681', '151682', '152130', '152152', '152154', '152156', '152157', '152158', '152159', '152160', '152162', '152166', '152168', '152169', '152170', '152171', '152172', '152173', '152174', '152177', '152178', '152181', '157769', '160544', '162301', '162327', '168522', '169072', '177612', '185545', '187461', '190246', '190249', '191487', '191489', '206925', '207871', '210184', '210185', '210258', '210322', '210385', '210387', '210638', '210639', '210640', '210641', '210986', '211128', '211132', '211680', '212415', '212752', '214114', '230304', '233360', '233832', '233833', '234057', '234375', '235276', '235782', '235785', '235786', '237078', '237083', '237084', '237086', '237178', '249096', '257325', '264441', '264461', '264468', '264469', '264470', '264471', '264472', '264473', '264474', '264475', '264476', '264477', '264478', '264479', '264480', '264481', '264482', '264483', '264484', '266754', '266755', '267586', '267594', '268043', '268244', '268748', '268751', '268752', '268950', '269084', '269085', '269622', '269626', '270405', '271617', '271623', '271624', '271626', '271629', '271631', '271632', '271634', '271637', '271638', '271639', '271641', '271644', '271649', '271652', '271653', '271659', '271665', '271670', '271672', '273031', '274188', '274407', '274836', '275155', '278083', '278085', '278088', '278090', '278092', '278094', '278479', '278645', '280551', '284400', '284592', '284597', '284600', '284605', '284821', '285486', '286109', '287433', '292087', '296563', '296564', '296914', '309601', '313796', '314034', '314038', '9010', '18372', '19750', '22037', '22058', '22693', '22943', '22970', '22979', '22980', '22989', '22992', '22996', '22999', '23001', '23466', '23470', '23471', '23484', '23488', '23489', '23491', '23492', '23493', '23500', '23514', '34290', '34292', '35783', '36361', '40042', '40046', '43651', '43652', '46919', '46973', '46998', '47134', '48199', '48208', '54740', '54766', '54772', '56696', '56878', '58578', '58587', '58592', '58595', '58596', '58598', '58599', '58600', '58601', '58604', '58614', '58617', '58619', '58630', '58643', '58644', '58663', '58667', '58677', '58703', '58707', '58712', '58717', '58726', '58734', '58735', '58737', '58740', '58749', '58754', '58763', '58780', '58784', '58801', '58825', '58837', '58857', '58866', '60212', '60215', '63211', '63419', '63619', '63687', '84778', '84793', '85651', '86236', '86237', '86240', '86244', '86246', '88127', '88129', '96792', '97349', '98240', '107421', '118560', '118890', '119010', '121179', '121180', '121362', '123619', '123947', '127086', '130354', '138343', '138344', '138779', '150349', '150434', '150442', '150443', '150477', '166880', '170666', '173980', '184578', '184579', '184580', '184838', '210182', '210183', '210326', '210734', '210908', '210912', '210914', '210916', '210918', '210919', '211043', '211208', '211662', '213661', '214460', '214461', '217166', '230307', '230308', '230312', '234413', '234415', '235781', '235784', '237076', '237832', '237933', '240453', '258616', '263886', '263887', '264754', '264756', '264757', '267599', '268190', '268191', '268193', '268633', '268737', '268740', '268742', '268744', '268746', '268951', '269511', '269512', '269965', '270461', '270471', '270474', '271612', '271615', '271616', '271621', '271635', '271646', '271647', '271656', '271657', '271662', '271664', '271666', '271674', '271675', '273217', '273218', '273219', '273792', '273793', '274153', '274154', '274155', '274156', '274849', '274850', '274851', '274852', '274999', '275157', '275401', '275652', '275655', '275662', '275663', '275841', '275842', '275843', '275844', '278053', '278082', '278084', '278086', '278087', '278089', '278091', '278236', '279450', '279452', '279456', '279459', '279460', '279740', '279742', '279743', '279744', '279748', '280547', '280548', '280549', '280550', '280552', '284819', '284994', '284995', '285491', '288712', '301219', '301221', '301222', '301223', '301224', '301225', '301226', '301227', '301228', '301229', '301231', '301232', '301233', '301234', '301235', '301236', '301239', '301242', '301258', '301259', '301260', '301262', '301266', '301267', '301269', '301270', '301272', '301273', '301274', '301276', '301278', '301280', '308515', '308527', '309306', '309487', '309489', '309490', '8850', '8858', '9532', '9536', '9538', '9547', '9900', '14928', '14932', '16016', '16293', '18091', '18092', '18374', '19047', '19830', '19837', '20185', '22700', '22720', '22973', '27312', '27313', '27339', '28293', '30143', '30152', '30984', '31190', '31195', '31196', '31198', '31200', '31206', '34593', '35607', '36332', '36800', '43748', '43749', '43750', '43751', '46908', '47085', '47133', '47136', '47137', '47138', '47201', '47214', '48439', '50943', '50963', '51655', '52638', '52640', '52647', '53089', '53090', '57587', '57594', '58093', '58579', '58591', '58609', '58616', '58618', '58623', '58631', '58638', '58665', '58678', '58684', '58689', '58690', '58694', '58699', '58705', '58706', '58708', '58709', '58710', '58713', '58719', '58720', '58731', '58733', '58761', '58779', '58802', '58803', '58835', '58838', '58839', '58843', '58844', '58850', '58881', '58893', '58912', '58921', '58922', '58939', '58941', '59904', '59906', '59907', '59908', '59909', '59910', '59911', '59912', '59915', '59917', '59918', '59919', '59922', '59923', '63860', '63881', '63882', '64090', '64103', '64104', '64998', '65001', '65004', '65009', '65291', '66851', '66853', '66854', '69259', '70760', '70932', '70933', '71329', '73086', '73481', '74541', '74717', '74904', '74905', '74908', '75925', '75932', '75933', '77342', '77706', '77718', '77719', '77784', '79044', '85056', '85543', '85548', '85551', '86241', '87073', '88098', '88101', '88131', '88134', '90084', '90198', '90241', '90548', '96937', '97686', '100047', '101098', '109797', '109798', '116685', '116687', '116688', '116689', '116690', '116717', '118462', '119008', '119064', '120244', '120426', '120427', '121663', '123152', '123580', '125115', '127071', '131381', '134094', '134675', '134887', '135100', '135265', '136057', '138300', '138301', '138777', '138778', '139095', '145013', '146859', '150351', '151683', '151684', '152161', '152163', '154955', '157421', '158544', '158869', '160170', '164006', '166123', '166125', '166135', '166151', '166152', '166153', '167155', '167579', '168009', '168189', '169050', '170667', '170668', '170669', '170670', '170711', '173654', '173962', '176257', '176743', '184576', '184836', '184837', '184910', '186007', '188421', '190245', '191603', '192789', '194663', '194744', '205662', '206866', '210257', '210915', '210917', '211209', '212414', '212416', '212417', '212418', '213662', '214112', '214116', '214117', '219743', '219748', '226179', '226180', '226181', '226182', '226183', '226184', '226185', '226348', '226699', '226701', '226709', '226895', '226899', '227092', '229135', '230313', '235288', '237079', '237080', '237081', '237082', '238703', '238704', '239041', '241798', '243234', '247115', '248039', '249441', '250483', '252800', '252805', '252806', '252807', '252915', '253387', '254903', '254904', '254905', '254906', '254907', '254908', '254912', '254913', '254914', '255097', '255543', '255544', '255545', '255546', '255631', '255685', '255687', '255688', '255691', '255693', '255695', '255696', '255698', '255703', '255705', '255706', '255711', '255713', '255717', '255745', '256645', '256646', '257318', '257319', '257329', '257454', '257651', '257686', '259156', '260505', '260508', '260509', '260510', '260511', '260515', '260516', '260517', '260519', '260521', '260522', '260525', '260540', '261664', '261665', '261667', '261668', '263984', '263985', '263986', '264164', '264165', '264167', '264343', '264345', '264346', '264347', '264357', '264363', '264365', '264367', '264368', '264369', '264371', '264373', '264374', '264375', '264376', '264380', '264381', '264382', '264383', '264384', '264386', '264387', '264434', '264435', '264436', '264437', '264438', '264439', '264440', '264445', '264449', '264450', '264743', '264744', '264748', '264755', '264758', '265070', '265798', '265799', '265800', '265801', '265802', '265808', '266013', '266673', '266678', '266681', '266689', '266692', '266694', '266745', '266747', '266756', '266757', '267420', '267423', '267424', '267426', '267527', '267587', '267598', '267601', '267637', '267811', '267813', '267838', '267947', '267948', '267953', '267954', '267955', '267956', '268171', '268172', '268173', '268174', '268175', '268176', '268177', '268178', '268179', '268180', '268181', '268186', '268187', '268188', '268189', '268192', '268221', '268223', '268238', '268240', '268242', '268260', '268261', '268358', '268359', '268360', '268414', '268480', '268481', '268532', '268533', '268534', '268535', '268632', '268635', '268666', '268681', '268747', '268749', '268750', '268754', '268830', '268856', '268857', '268859', '268860', '268984', '269627', '269707', '269708', '269968', '269970', '270072', '270470', '270475', '270670', '270674', '270682', '270814', '270815', '270816', '270817', '270818', '270819', '270821', '270822', '270823', '270824', '270825', '270890', '270914', '270916', '270918', '270919', '270922', '270923', '270924', '270925', '271145', '271603', '271619', '271630', '271636', '271645', '271648', '271658', '271663', '271668', '271671', '272259', '272275', '272443', '272445', '272446', '272447', '272594', '272597', '272601', '272794', '272797', '273094', '273095', '273096', '273361', '273362', '273363', '273364', '273403', '273404', '273405', '273541', '273542', '273543', '273544', '273646', '273647', '273648', '273649', '273650', '273651', '273710', '273711', '273712', '273713', '273714', '273715', '273716', '273717', '273718', '273719', '273720', '273721', '273821', '273878', '273882', '273884', '273890', '273896', '273902', '273930', '273944', '274040', '274400', '274401', '274402', '274403', '274404', '274405', '275000', '275653', '275695', '276505', '276584', '276589', '276595', '276601', '277027', '277030', '277031', '277032', '277226', '277227', '277228', '277229', '277240', '277241', '277242', '277244', '277245', '277246', '277306', '277907', '278093', '278647', '279389', '280465', '280466', '280467', '280468', '280537', '280539', '281450', '281451', '281547', '282349', '282360', '282538', '283676', '284359', '284367', '284370', '284393', '284823', '285293', '285359', '285360', '285361', '285362', '285367', '285372', '285374', '285382', '285383', '285384', '285485', '285487', '285488', '285865', '285880', '285882', '286005', '286006', '286008', '286016', '286019', '286020', '286021', '286022', '286026', '286094', '286100', '286102', '288071', '288073', '288075', '288076', '288116', '288124', '288125', '288126', '288128', '288130', '288131', '288132', '288133', '288134', '288136', '288138', '288215', '288218', '288219', '288220', '288221', '288223', '288267', '288637', '288639', '288641', '288643', '288645', '288649', '288814', '288815', '289053', '289054', '289069', '289070', '289178', '289356', '289360', '289363', '289365', '289855', '290192', '290193', '290194', '290218', '290219', '290220', '290676', '290684', '290776', '290907', '290908', '290916', '290917', '290918', '290919', '290921', '290927', '290929', '290930', '290933', '290935', '290937', '290938', '290939', '290940', '291298', '292025', '292027', '292062', '292063', '292064', '292065', '292066', '292067', '292068', '292070', '292076', '292085', '292086', '292126', '292132', '293846', '293847', '294454', '294455', '294456', '294458', '294459', '294460', '294461', '294462', '294463', '294636', '294871', '294898', '294901', '294942', '294943', '294944', '295765', '295766', '296257', '296258', '296261', '296453', '296637', '296653', '296711', '296714', '296716', '296718', '296719', '296720', '296721', '296722', '296724', '296728', '296729', '296730', '296775', '296797', '296798', '296800', '296801', '296804', '296807', '296809', '296810', '296813', '296937', '296941', '296942', '296944', '296946', '296994', '296995', '296996', '296997', '296998', '296999', '297000', '297001', '297066', '297070', '297316', '297608', '297612', '297619', '297736', '300805', '300806', '300807', '300808', '300809', '300810', '300811', '300812', '300813', '300814', '300815', '300833', '300834', '300858', '300862', '301263', '301268', '302329', '302709', '302710', '302711', '302712', '302713', '302714', '302715', '303629', '304854', '304855', '305044', '305045', '305046', '305048', '305049', '305053', '305054', '305055', '305554', '305799', '305857', '305898', '306742', '306743', '307525', '307527', '307725', '307727', '307729', '307791', '307792', '307799', '307808', '307809', '307877', '307878', '307879', '307880', '307881', '308002', '308006', '308009', '308015', '308017', '308018', '308022', '308045', '308062', '308065', '308068', '308069', '308071', '308076', '308079', '308242', '308471', '308472', '308473', '308474', '308475', '308476', '308534', '308583', '308584', '308866', '308922', '308924', '308925', '308926', '308928', '308930', '308931', '308947', '309122', '309125', '309132', '309144', '309145', '309146', '309148', '309149', '309166', '309167', '309296', '309299', '309302', '309303', '309307', '309309', '309310', '309311', '309316', '309320', '309321', '309332', '309343', '309344', '309346', '309347', '309356', '309357', '309358', '309366', '309368', '309369', '309371', '309372', '309375', '309377', '309380', '309384', '309385', '309409', '309410', '309414', '309415', '309424', '309468', '309561', '309562', '309563', '309564', '309565', '309566', '309568', '309570', '309572', '309591', '309592', '309593', '309595', '309596', '309602', '310260', '310327', '310974', '313754', '313788', '313792', '313794', '313798', '313799', '313801', '313804', '313805', '313806', '313807', '313808', '313809', '313811', '313812', '313814', '313815', '313818', '313819', '313822', '313824', '313832', '313833', '313836', '313841', '313844', '313846', '313849', '313853', '313857', '314085', '314157', '322607', '322608', '322678', '323004', '323005', '323006', '323007', '323008', '323010', '323015', '323018', '323019', '323020', '324233', '330505', '330506', '331682', '8862', '10082', '10096', '18353', '19751', '22873', '22874', '23305', '23306', '23307', '30991', '39655', '39656', '39665', '45918', '48745', '52328', '56970', '58095', '58376', '58534', '58535', '58655', '58656', '58661', '58673', '58674', '58714', '58827', '58884', '58917', '58926', '58927', '58932', '58934', '60840', '61070', '65002', '65659', '65662', '67965', '70236', '70242', '71303', '71340', '71341', '71344', '71364', '71365', '71366', '71373', '71374', '71392', '71394', '71395', '71397', '71404', '71405', '71406', '71407', '71408', '71409', '71413', '71415', '71416', '71433', '71444', '71949', '71967', '75934', '84738', '85396', '99705', '115096', '117515', '117516', '118373', '120224', '122963', '129817', '130025', '130026', '130027', '130029', '130030', '130031', '130160', '130317', '130353', '130677', '130678', '130680', '130732', '139091', '150945', '152042', '152164', '152175', '152176', '152179', '152203', '152576', '171321', '171356', '171358', '171359', '171373', '173879', '175656', '177614', '183160', '184243', '185411', '191110', '191112', '198128', '198129', '198258', '199931', '199932', '200155', '200212', '200214', '200215', '200326', '200327', '200328', '200329', '200330', '200331', '200332', '200387', '200388', '200389', '200392', '200393', '200483', '200484', '200485', '212753', '213995', '230301', '237085', '247879', '248040', '264467', '264750', '264751', '264752', '264753', '266009', '266011', '266012', '266748', '266750', '266751', '267589', '267596', '267600', '268185', '268738', '268739', '268743', '268745', '270641', '271642', '271655', '271660', '271661', '272888', '272889', '272890', '273136', '273138', '276808', '279746', '282912', '285869', '285870', '287788', '287793', '287794', '287795', '287853', '290197', '290198', '296561', '296562', '297615', '313754', '313898', '15666', '15670', '15671', '15684', '15685', '15686', '15690', '15692', '15700', '15703', '15704', '15705', '15707', '15712', '15713', '15714', '15716', '15724', '15725', '15730', '15734', '15739', '15740', '15741', '15742', '15779', '48336', '105609', '118521', '156970', '169330', '169644', '175450', '175453', '175454', '175457', '175461', '193417', '193419', '193420', '193421', '193422', '193423', '193444', '193445', '193446', '193447', '193448', '193450', '193451', '193452', '193453', '194338', '194339', '194340', '195059', '195060', '196244', '203291', '210965', '210966', '210967', '228969', '253083', '253186', '253187', '253188', '253190', '258376', '258377', '258379', '258380', '258381', '258382', '258383', '258384', '258778', '258779', '258780', '258781', '258782', '258783', '258784', '258785', '258786', '258804', '258805', '258871', '262731', '263042', '263043', '263044', '263045', '263046', '263047', '263048', '263049', '263050', '263051', '263052', '263053', '263054', '263055', '263056', '263057', '263058', '263059', '263060', '263061', '263062', '263063', '263066', '263067', '263068', '263069', '263070', '263071', '263072', '263073', '263074', '263075', '263076', '263077', '263081', '263082', '263083', '263084', '263085', '263086', '263087', '263088', '263089', '263090', '263091', '263092', '263093', '263094', '263095', '263096', '263097', '263098', '263100', '263102', '263103', '263104', '263105', '263114', '263116', '263163', '263953', '266226', '266227', '269831', '271613', '271614', '272197', '272198', '272199', '272200', '272201', '272202', '272204', '272205', '272435', '272436', '272437', '272438', '272661', '272663', '272664', '272665', '273692', '277809', '298500', '299188', '9821', '10172', '10186', '10188', '10190', '10193', '10195', '19572', '19573', '19574', '19576', '35599', '35611', '47147', '59927', '68995', '74335', '84641', '96793', '96794', '96795', '96796', '120465', '123049', '130091', '148914', '179284', '179285', '179286', '179863', '181457', '185064', '185066', '220332', '226707', '226708', '264372', '264785', '267823', '268369', '268636', '268649', '271620', '271622', '271651', '281947', '281990', '296803', '296805', '296806', '296808', '296947', '296949', '301279', '309124', '320932', '320935', '320939', '320941', '321232', '9272', '10857', '10859', '10863', '10896', '10897', '10908', '10910', '10916', '10917', '10921', '10922', '10924', '10926', '10930', '10932', '10936', '10938', '10939', '10941', '10942', '10945', '10946', '10948', '10950', '10952', '10956', '10958', '10961', '10963', '10965', '10966', '10972', '10975', '10977', '10980', '10982', '10983', '10985', '10987', '10988', '10989', '10992', '10997', '11670', '11848', '11850', '11855', '11857', '11858', '17357', '22754', '23004', '28351', '32851', '32864', '32865', '32866', '32867', '34351', '35612', '48131', '48247', '48524', '48656', '49406', '53042', '53138', '53139', '53140', '53142', '53145', '53146', '57667', '57668', '57823', '60784', '60785', '60786', '60789', '62365', '62376', '62392', '62393', '62395', '72611', '72612', '76766', '76769', '76771', '76772', '79482', '84325', '84334', '84335', '85121', '85122', '85185', '85186', '85187', '87226', '88100', '116681', '116682', '116684', '128965', '132974', '133139', '134085', '134247', '137582', '138136', '138432', '149291', '151564', '151597', '154910', '154978', '154979', '156906', '165363', '169120', '169121', '170653', '173867', '174847', '184982', '184983', '184984', '186392', '188588', '195284', '198281', '198283', '199026', '200403', '202320', '202321', '204421', '210656', '210733', '212189', '213450', '234746', '247315', '247316', '247317', '247318', '247319', '248285', '257033', '257034', '257035', '257037', '257039', '257040', '257041', '257091', '263944', '263945', '263947', '263948', '264196', '264197', '267209', '267211', '267212', '267214', '269636', '270715', '270716', '270717', '271628', '272240', '272248', '272249', '272250', '272251', '272252', '272253', '272254', '272735', '272741', '272743', '274031', '274032', '275002', '277808', '277846', '277847', '277848', '277849', '277850', '277946', '278246', '278253', '279310', '279311', '279312', '279313', '279314', '279315', '279316', '281217', '281223', '282634', '282649', '283024', '283025', '283026', '283027', '283946', '284075', '284078', '284081', '284356', '285014', '285015', '285016', '285017', '285019', '285385', '285386', '285387', '285388', '285389', '287957', '287959', '287960', '291072', '291082', '291084', '291116', '291117', '291119', '291120', '291121', '291441', '291442', '291656', '291657', '291658', '291659', '292113', '292114', '292115', '294679', '296666', '297611', '309604', '309606', '309607', '309608', '309611', '309612', '309613', '309614', '313848', '313852', '313855', '15785', '19118', '50947', '61301', '175466', '175468', '175474', '175477', '175494', '175497', '193471', '193482', '193485', '193491', '282278', '282279', '282805', '282806', '282807', '282808', '282809', '24297', '24300', '24301', '24309', '24321', '24322', '24334', '24340', '52328', '124015', '124016', '124017', '134674', '176094', '252292', '256301', '256302', '256303', '256304', '260578', '263992', '267364', '267365', '267366', '267377', '272237', '272238', '278208', '282544', '282545', '298981', '298982', '28502', '28507', '28530', '115085', '170318', '191062', '193469', '193470', '234376', '255037', '255093', '256476', '263106', '263107', '263108', '263109', '263110', '263111', '263113', '263954', '267967', '268537', '269947', '272210', '272211', '280390', '9534', '15678', '15679', '15694', '15695', '15696', '15697', '15701', '15715', '15721', '15727', '30134', '35428', '60684', '60685', '60686', '60687', '60706', '60729', '60742', '61295', '77700', '106148', '152074', '152075', '152076', '152077', '162850', '162851', '162852', '162853', '162854', '175498', '193449', '193456', '224430', '226589', '232958', '261356', '262296', '263064', '263065', '263078', '263079', '263080', '263112', '263131', '263142', '263150', '263157', '263158', '263170', '263171', '263172', '264324', '264591', '293189', '293190', '293191', '293192', '293193', '293194', '293195', '293196', '293197', '293198', '293199', '293200', '293201', '293202', '302666', '302667', '302668', '302669', '302670', '302671', '302672', '302673', '302674', '302675', '305512', '309305', '9909', '22872', '22880', '23308', '23365', '26432', '39186', '39666', '42910', '42912', '42913', '42914', '42915', '42916', '48113', '48187', '48194', '48195', '48196', '48206', '48207', '48280', '48328', '48343', '48344', '48345', '48387', '48493', '48494', '48495', '48496', '48504', '48511', '48512', '48513', '48514', '48516', '48517', '48527', '48585', '48657', '48659', '48660', '48661', '48662', '48663', '48664', '48665', '48667', '48701', '48718', '48721', '48723', '48724', '48733', '48734', '48737', '48738', '48739', '48740', '48741', '48742', '48746', '48747', '48748', '48749', '48750', '48751', '48752', '48754', '48755', '48756', '48758', '48759', '48760', '48761', '56971', '56973', '58288', '58289', '58290', '58308', '58313', '58314', '58328', '58336', '58339', '58340', '58343', '58352', '58353', '58533', '58538', '58540', '60897', '61132', '63835', '63839', '65647', '65654', '65658', '71347', '71402', '71427', '71945', '71977', '77209', '78852', '79040', '85724', '85725', '85788', '87286', '89875', '99823', '99824', '106933', '112730', '113194', '113195', '113969', '114278', '115481', '116756', '117721', '127087', '127088', '127103', '129819', '130340', '130674', '130692', '130693', '130694', '130695', '130696', '130697', '130698', '130699', '130700', '130728', '130756', '136090', '136091', '137833', '144590', '144591', '146858', '147489', '149843', '151136', '151750', '151751', '151752', '151753', '151778', '151779', '151780', '151781', '151861', '152043', '152057', '152058', '152059', '152060', '152182', '152183', '152232', '152577', '157679', '157680', '157681', '157682', '157683', '157684', '157785', '157786', '157787', '157788', '157823', '157824', '157825', '157826', '159749', '164637', '165409', '166911', '167485', '171319', '171320', '173619', '173888', '173893', '175657', '176729', '178420', '178424', '178699', '178917', '178920', '179487', '180644', '182130', '183158', '184330', '184331', '184333', '184363', '185463', '186075', '186944', '187961', '188072', '188074', '188075', '188645', '192187', '192480', '192785', '196197', '196198', '196199', '196234', '196235', '196236', '196237', '196238', '196243', '196245', '196246', '196247', '196266', '196267', '196268', '196269', '196270', '196271', '196273', '196274', '196275', '196460', '196461', '196688', '196689', '196972', '196974', '197055', '197202', '197203', '197204', '197205', '197206', '197207', '197208', '197376', '197460', '197755', '197757', '197758', '197760', '198126', '198131', '198132', '198147', '198367', '198458', '198459', '198460', '198914', '199184', '199185', '199268', '199270', '199935', '199936', '200201', '200203', '200205', '200208', '200210', '200211', '200223', '200415', '200416', '200417', '200418', '200550', '202210', '202276', '202277', '202280', '224596', '224602', '224605', '224607', '224608', '224610', '224611', '224612', '224613', '224615', '225931', '235323', '247491', '247551', '247553', '247880', '247881', '247883', '251491', '251492', '254853', '254855', '254858', '254859', '262639', '264815', '266232', '267532', '267533', '267547', '267554', '267620', '267660', '273125', '279443', '284158', '290206', '290222', '293848', '293849', '296506', '299187', '304696', '305024', '313757', '313861', '313891', '319888', '320018', '320074', '19571', '19575', '37149', '37151', '37158', '50198', '50207', '50208', '50210', '50211', '50212', '52072', '85932', '135489', '164485', '164486', '164487', '164489', '164491', '164492', '164500', '164568', '164571', '164605', '164606', '190230', '227199', '229134', '264462', '264465', '265685', '267809', '271627', '307568', '307570', '309162', '28001', '51211', '51223', '51224', '51276', '51281', '51311', '51327', '51328', '51337', '51339', '51351', '51359', '51412', '51414', '51416', '51417', '51420', '51425', '51430', '51431', '51434', '51435', '51440', '51469', '51470', '51475', '51483', '51492', '51514', '51552', '51560', '51608', '51610', '51620', '51651', '51652', '51662', '51666', '73082', '74324', '74329', '74331', '74332', '74953', '85909', '85911', '85916', '88121', '97348', '98236', '118252', '118253', '118254', '119033', '119034', '119035', '134093', '134098', '134099', '161318', '169473', '169474', '170628', '173897', '176093', '180267', '181792', '194290', '200007', '200722', '202391', '202396', '257260', '257262', '257263', '257343', '257344', '257924', '259687', '259688', '267293', '267294', '267295', '267296', '267298', '267299', '267498', '267505', '267506', '267515', '267516', '267517', '267518', '267558', '267559', '267560', '267561', '267562', '267563', '267564', '267565', '267618', '267619', '268052', '268644', '268645', '268646', '268647', '268648', '269034', '269934', '269935', '269936', '269938', '271846', '271939', '272122', '272123', '272124', '272125', '272126', '272128', '272129', '272130', '272131', '272134', '272135', '272136', '272137', '272139', '272140', '272141', '272142', '273864', '278209', '278210', '282541', '282542', '282680', '283951', '285634', '291994', '302394', '307310', '307456', '307457', '307458', '307459', '307468', '307469', '307470', '307471', '52251', '52258', '52259', '52263', '52264', '52267', '52268', '52279', '52280', '52282', '52284', '52285', '52286', '52289', '52290', '52293', '52294', '52297', '52305', '52309', '52310', '52311', '52313', '52318', '52319', '52322', '52327', '52330', '52331', '52339', '52343', '52344', '52354', '52355', '52356', '52357', '52358', '52359', '52360', '52364', '52365', '52366', '52367', '52368', '52369', '52370', '52371', '52475', '62373', '62385', '62390', '62394', '75854', '85935', '88122', '97302', '98246', '118424', '118425', '123856', '124861', '130219', '130220', '130221', '130755', '130858', '131102', '131109', '133559', '146553', '146554', '146555', '146710', '148954', '151565', '166454', '166457', '195162', '200006', '227398', '227400', '235294', '235296', '242238', '251317', '292016', '292020', '292021', '292023', '309598', '309600', '309605', '313901', '313903', '320928', '19594', '51490', '51491', '52410', '52411', '52420', '52421', '52426', '52427', '52428', '52430', '52433', '52436', '52437', '52441', '52447', '52458', '52461', '52466', '52467', '52468', '52470', '52471', '85939', '85945', '85947', '85950', '90643', '129617', '133965', '134092', '134136', '134137', '180452', '205404', '205405', '205929', '237203', '237836', '237837', '237839', '238111', '238119', '242204', '251320', '252821', '252822', '252823', '252824', '252827', '257455', '260523', '260524', '264247', '264249', '264250', '264251', '264252', '264253', '264255', '264256', '264257', '264258', '264512', '269927', '269928', '269929', '269930', '272224', '272225', '272226', '272227', '272228', '272229', '272230', '272231', '272232', '272233', '272235', '273868', '273869', '273870', '275516', '275517', '275519', '275521', '277650', '277651', '277652', '277653', '277654', '277655', '277656', '277657', '277773', '277844', '277845', '278054', '278178', '278179', '279002', '279970', '279971', '279972', '279973', '279974', '279975', '280211', '280213', '280214', '280215', '280217', '280218', '285701', '285717', '285728', '286313', '286322', '286326', '286327', '286328', '286330', '286342', '286343', '289051', '290610', '296398', '296583', '296585', '296993', '297004', '297639', '298400', '298401', '305708', '305709', '305776', '305800', '309503', '321131', '322936', '324354', '324356', '328291', '330321', '330359', '9516', '48133', '48134', '48135', '48181', '48520', '58773', '114353', '117629', '117630', '130108', '130553', '248036', '249651', '249654', '250366', '250803', '251161', '251162', '251310', '251311', '251925', '252919', '252920', '253218', '253835', '253964', '254718', '254759', '254760', '254823', '254841', '254889', '255542', '256773', '258292', '258613', '258842', '259580', '260502', '260503', '264233', '264915', '268253', '268497', '270853', '276645', '276646', '284373', '289265', '289266', '289268', '289269', '289272', '289366', '289367', '289369', '289494', '291399', '296776', '296812', '15774', '15775', '48111', '48112', '48118', '48154', '48232', '48233', '48234', '48257', '48262', '48272', '48312', '48313', '48391', '48393', '48396', '48400', '48405', '48584', '48694', '48700', '48762', '48765', '48776', '48778', '48779', '48783', '48784', '48798', '48799', '50199', '71039', '71040', '71042', '88108', '90481', '144152', '144153', '144154', '144156', '144158', '144159', '146294', '146455', '160603', '163532', '167775', '180636', '193454', '193455', '193457', '193467', '193468', '204916', '204917', '204919', '205052', '205053', '212382', '227201', '240212', '252411', '252808', '256221', '256222', '256289', '256294', '258777', '263099', '263101', '263159', '263160', '263161', '263162', '263164', '263165', '263166', '263167', '263168', '263775', '263776', '267399', '267403', '267404', '267536', '267787', '267798', '267799', '267935', '267936', '268064', '268065', '273398', '273399', '273400', '273553', '273555', '273556', '274848', '275848', '275893', '275895', '275897', '275900', '275902', '277625', '278194', '278195', '278553', '279673', '280391', '280392', '280393', '280394', '280395', '287212', '307530', '307533', '314156', '314158', '314160', '314163', '48057', '48108', '48152', '48161', '48235', '48269', '48276', '48277', '48281', '48285', '48293', '48314', '48384', '48398', '48447', '48449', '48450', '48451', '48452', '48453', '48454', '48455', '48458', '48460', '48461', '48463', '48780', '48782', '85790', '137571', '138298', '157706', '164566', '178606', '180638', '180639', '182857', '186229', '204920', '207886', '210642', '210643', '210644', '210645', '240215', '240219', '243231', '243232', '253738', '267401', '273401', '273402', '275847', '275849', '275868', '275891', '275892', '278193', '282578', '282579', '290673', '306485', '307531', '33350', '35256', '35937', '40564', '48301', '48539', '48542', '48546', '48562', '48570', '48580', '48582', '49662', '55423', '68160', '119377', '125016', '211670', '226118', '226127', '240189', '250584', '250585', '250586', '255649', '267349', '269920', '279713', '309603', '317548', '319556', '57993', '58028', '58030', '58051', '58055', '58058', '58082', '58083', '132556', '184252', '194314', '234748', '278180', '278181', '278182', '286323', '286324', '286325', '297724', '297739', '300826', '320117', '320167', '48480', '48483', '60671', '60678', '60679', '60694', '60699', '60700', '60702', '60703', '60715', '60721', '60722', '60727', '60732', '60741', '170316', '170317', '170542', '187058', '247678', '258385', '258386', '258387', '258388', '258389', '258390', '258391', '258392', '258393', '258394', '258395', '258396', '258787', '258788', '258789', '258790', '258791', '258793', '258794', '258795', '258796', '258797', '258798', '258799', '261357', '263117', '263118', '263119', '263120', '263121', '263122', '263123', '263125', '263126', '263128', '263129', '263130', '263132', '263133', '263134', '263135', '263136', '263138', '263139', '263140', '263143', '263144', '263145', '263146', '263147', '263148', '263149', '263151', '263152', '263153', '263154', '288181', '313856', '19121', '34438', '50961', '61309', '106569', '106570', '106571', '109313', '109315', '109316', '122921', '122922', '122923', '123328', '123329', '123330', '130744', '130745', '130746', '147769', '147770', '147771', '174838', '174839', '174840', '175445', '175446', '175447', '193477', '193483', '193486', '198685', '198686', '198687', '206039', '206040', '206041', '213130', '213131', '213132', '219754', '219755', '226772', '226773', '226774', '227518', '234692', '252940', '271342', '272551', '299184', '299186', '48219', '66112', '66190', '66191', '66192', '117457', '186440', '273155', '273157', '273158', '277252', '279265', '279266', '68994', '69024', '69025', '69089', '69093', '69094', '69096', '69097', '69128', '69139', '69140', '69141', '69142', '69150', '69151', '69158', '69161', '69162', '69164', '69166', '69170', '69171', '69172', '69175', '69176', '69177', '69178', '69179', '69254', '69260', '69278', '69283', '69284', '69289', '69290', '69291', '69298', '69301', '69305', '69307', '69308', '69309', '69310', '69312', '69316', '69317', '69321', '69323', '69347', '69351', '69352', '69353', '69354', '83717', '83719', '83721', '83723', '88136', '120625', '129551', '130106', '130107', '130736', '132709', '135269', '135270', '135271', '151239', '151240', '167187', '167190', '167288', '167289', '167291', '170040', '170065', '175528', '175891', '178306', '180792', '184484', '184485', '184486', '184487', '187458', '194286', '194312', '194313', '199262', '199282', '200220', '202397', '210767', '210770', '226176', '226712', '229316', '229317', '234412', '234733', '235126', '235127', '237204', '237209', '238897', '247307', '249617', '249659', '259880', '259881', '259884', '263993', '264012', '264016', '264501', '264504', '264505', '264507', '264510', '264511', '267302', '267303', '267304', '269931', '269932', '269933', '270160', '270695', '272234', '272236', '273865', '273866', '273867', '282539', '282558', '282559', '282561', '282562', '283979', '284337', '284353', '284358', '286117', '286433', '286434', '288135', '288137', '288140', '290284', '290398', '290399', '290435', '290443', '290500', '291208', '291334', '291335', '291336', '292229', '293301', '297028', '297037', '297042', '297052', '297064', '298273', '300100', '308705', '308715', '313391', '313893', '313895', '313896', '313897', '313899', '313900', '319712', '319713', '319714', '319809', '320931', '320933', '321227', '324226', '8947', '8952', '8956', '8964', '9711', '9712', '9713', '9714', '9717', '9722', '35608', '73022', '73144', '73145', '73146', '73147', '73155', '73166', '73207', '73208', '73216', '73218', '73227', '73234', '83885', '87210', '99569', '99570', '99572', '99678', '101019', '101021', '101022', '117800', '123050', '123051', '123052', '151648', '151649', '151650', '151651', '151652', '151732', '151809', '151810', '151811', '151907', '151912', '154029', '166991', '175660', '182929', '204320', '204321', '210648', '210655', '214175', '220323', '226173', '237960', '248276', '248277', '248279', '248281', '248282', '248283', '248290', '251371', '252538', '258599', '258601', '258602', '258606', '258607', '258608', '258609', '258611', '260138', '264423', '264424', '265222', '265223', '265304', '265307', '265312', '265415', '265416', '265417', '265418', '265419', '265420', '272239', '272241', '272242', '272243', '272244', '272245', '272246', '272255', '275001', '279318', '279319', '281978', '283630', '283631', '283632', '283633', '283634', '283635', '292121', '308843', '309544', '309545', '309546', '309547', '309548', '309550', '309551', '309552', '309553', '309554', '309555', '309556', '309577', '309578', '309579', '309580', '309581', '309582', '309583', '309586', '309587', '309588', '309589', '313858', '313860', '8955', '8960', '8963', '73148', '73158', '73181', '73184', '73197', '73209', '73213', '73229', '73250', '73251', '87209', '88140', '90035', '99571', '117391', '118085', '129748', '130218', '148773', '169712', '214177', '214333', '220313', '220320', '220325', '220328', '220437', '258605', '258610', '264422', '11783', '11784', '11787', '84358', '117522', '117523', '190224', '256628', '256629', '264464', '271643', '307882') and source_store_id = 116 and date(order_date) between to_date('{start_date}', 'YYYY-MM-DD') and to_date('{end_date}', 'YYYY-MM-DD')
    group by source_product_id;
"""

cur.execute(query)

print('Executed')

df = pd.DataFrame(cur.fetchall())

cur.close()
conn.close()

Executed


In [360]:
df_wnodes = pd.read_csv('single.csv', delimiter=';', header=None, encoding='windows-1251')
df_wnodes.columns = ['code', 'unit', 'name', 'weight', 'group']
df_wnodes

,code,unit,name,weight,group
0,15799,кг.,ГОЛЕНЬ ЦБ АЛЕЛЬ КГ,1.00,"Kуры(включая цыплят), свежий и охл., части ту..."
1,30694,кг.,КОСТИ КУРИНЫЕ/ОКОРОЧКА,1.00,"Kуры(включая цыплят), свежий и охл., части ту..."
2,34421,кг.,ГОЛЕНЬ Ц/Б С/ПР КГ,1.00,"Kуры(включая цыплят), свежий и охл., части ту..."
3,34423,кг.,МЯКОТЬ Ц/Б С/ПР КГ,1.00,"Kуры(включая цыплят), свежий и охл., части ту..."
4,34426,кг.,БЕДРО Ц/Б С/ПР КГ,1.00,"Kуры(включая цыплят), свежий и охл., части ту..."
...,...,...,...,...,...
4434,256628,кг.,ВАРЕНИКИ С КУРАГОЙ ОТВАРНЫЕ,1.00,Вареники
4435,256629,кг.,ВАРЕНИКИ С ЧЕРНОСЛИВОМ ОТВАРНЫЕ,1.00,Вареники
4436,264464,шт.,ВАРЕНИНИКИ С КАРТОФЕЛЕМ ОТВАРНЫЕ 200Г,0.30,Вареники
4437,271643,шт.,ДОСТАВКА ВАРЕНИКИ С КАРТОФЕЛЕМ ОТВАРНЫЕ 250Г,0.25,Вареники


In [361]:
df.columns = ['code', 'avg_price', 'quantity']
df

,code,avg_price,quantity
0,58712,1977.29,58.612
1,273718,500.00,8.000
2,273721,500.00,7.000
3,106568,2056.84,1460.960
4,58595,3740.75,5.414
...,...,...,...
467,69128,4400.00,3.308
468,106567,1210.27,274.527
469,278181,2800.00,3.490
470,10728,3213.38,3.010


In [362]:
df_wnodes['code'] = df_wnodes['code'].astype('str')
df['code'] = df['code'].astype('str')

In [363]:
df

,code,avg_price,quantity
0,58712,1977.29,58.612
1,273718,500.00,8.000
2,273721,500.00,7.000
3,106568,2056.84,1460.960
4,58595,3740.75,5.414
...,...,...,...
467,69128,4400.00,3.308
468,106567,1210.27,274.527
469,278181,2800.00,3.490
470,10728,3213.38,3.010


In [364]:
df_wnodes

,code,unit,name,weight,group
0,15799,кг.,ГОЛЕНЬ ЦБ АЛЕЛЬ КГ,1.00,"Kуры(включая цыплят), свежий и охл., части ту..."
1,30694,кг.,КОСТИ КУРИНЫЕ/ОКОРОЧКА,1.00,"Kуры(включая цыплят), свежий и охл., части ту..."
2,34421,кг.,ГОЛЕНЬ Ц/Б С/ПР КГ,1.00,"Kуры(включая цыплят), свежий и охл., части ту..."
3,34423,кг.,МЯКОТЬ Ц/Б С/ПР КГ,1.00,"Kуры(включая цыплят), свежий и охл., части ту..."
4,34426,кг.,БЕДРО Ц/Б С/ПР КГ,1.00,"Kуры(включая цыплят), свежий и охл., части ту..."
...,...,...,...,...,...
4434,256628,кг.,ВАРЕНИКИ С КУРАГОЙ ОТВАРНЫЕ,1.00,Вареники
4435,256629,кг.,ВАРЕНИКИ С ЧЕРНОСЛИВОМ ОТВАРНЫЕ,1.00,Вареники
4436,264464,шт.,ВАРЕНИНИКИ С КАРТОФЕЛЕМ ОТВАРНЫЕ 200Г,0.30,Вареники
4437,271643,шт.,ДОСТАВКА ВАРЕНИКИ С КАРТОФЕЛЕМ ОТВАРНЫЕ 250Г,0.25,Вареники


In [365]:
df_new = pd.merge(df, df_wnodes, on='code', how='inner')
for i in range(len(df_new)):
    print(df_new.loc[i, 'code'], df_new.loc[i, 'quantity'], df_new.loc[i, 'group'])
    break

58712 58.612 Блюда готовые овощные не замороженные


In [366]:
df_new

,code,avg_price,quantity,unit,name,weight,group
0,58712,1977.29,58.612,кг.,САЛАТ ИЗ СВЕКЛЫ С СЫРОМ,1.0,Блюда готовые овощные не замороженные
1,273718,500.00,8.000,шт.,"ШИЕ МОРСЫ/МОРС ИЗ ВИШН,500Г",0.5,Блюда готовые прочие
2,273721,500.00,7.000,шт.,"МОРС ИЗ КЛУБНИКИ ,500Г",0.5,Блюда готовые прочие
3,106568,2056.84,1460.960,кг.,ФИЛЕ Ц/Б КУС ВКУС ОХЛ КГ ЛОТОК,1.0,"Kуры(включая цыплят), свежий и охл., части ту..."
4,58595,3740.75,5.414,кг.,САЛАТ БАКЛАЖАНЫ ОСТРЫЕ,1.0,Блюда готовые овощные не замороженные
...,...,...,...,...,...,...,...
467,69128,4400.00,3.308,кг.,ТОРТ КИЕВСКИЙ,1.0,Торты свежие
468,106567,1210.27,274.527,кг.,ЧЕТВЕРТИНА Ц/Б КУС ВКУС ОХЛ КГ ЛОТОК,1.0,"Kуры(включая цыплят), свежий и охл., части ту..."
469,278181,2800.00,3.490,кг.,"РУЛЕТ АБРИКОСОВЫЙ, КГ",1.0,Рулеты
470,10728,3213.38,3.010,кг.,БРИЗОЛЬ КУРИНАЯ С МОРКОВЧЕЙ,1.0,Блюда готовые из мясо домашней птицы


In [367]:
df1 = pd.read_excel(r'C:\Users\Abdykarim.D\PycharmProjects\robot-1p\working_path\Торговый зал АСФ №1.xls')
df1.columns = ['code', 'wtf', 'name', 'unit', 'weight', 'cost', 'sold', 'sold_in_unit', 'price']
df1

,code,wtf,name,unit,weight,cost,sold,sold_in_unit,price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Дата выгрузки: с 01.01.2023 по 31.03.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Площадка: Торговый зал АСФ №1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Пользователь: Робот Ш..,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Альтернативная иерархия: Kуры(включая цыплят),...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
562,214333,107111100,ХЛЕБ СЕРЫЙ ФОРМОВОЙ 250 Г,шт.,0.25,125,1161,290.25,52979.01
563,264422,107111100,ХЛЕБ РЖАНОЙ ФОРМОВОЙ 300 Г,шт.,0.3,190,119,35.7,8836.7
564,ИТОГО: Хлеб ржаной,NaN,NaN,NaN,NaN,NaN,4674.82,1684.67,395114.27
565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [368]:
df1 = df1.dropna(how='all')
df1 = df1[df1[df1.columns[0]] != 'Код товара']
df1 = df1.dropna(subset=[df1.columns[2]])
df1 = df1.reset_index()
df1 = df1.drop(['index'], axis=1)
df1

,code,wtf,name,unit,weight,cost,sold,sold_in_unit,price
0,34426,101210532,БЕДРО Ц/Б С/ПР КГ,кг.,NaN,975,1.34,1.34,2227.28
1,34431,101210532,КУРИНЫЕ КОСТИ КГ,кг.,NaN,59,134.47,134.47,2392.39
2,34441,101210532,КУРИНЫЙ ФАРШ С/ПР КГ,кг.,NaN,2280,15.09,15.09,34847.4
3,106563,101210532,БЕДРО Ц/БРОЙЛЕРА КУС ВКУС ОХЛ КГ ЛОТОК,кг.,NaN,1583.63,1616.79,1616.79,2558040.1
4,106564,101210532,ГОЛЕНЬ Ц/Б КУС ВКУС ОХЛ КГ ЛОТОК,кг.,NaN,1888.01,1089.13,1089.13,1727337.23
...,...,...,...,...,...,...,...,...,...
467,117391,107111100,ХЛЕБ РЖАНОЙ 400 Г,шт.,0.4,300,141,56.4,16023.71
468,118085,107111100,ХЛЕБ ЛИТОВСКИЙ 400 Г,шт.,0.4,305,304,121.6,37492.32
469,169712,107111100,ХЛЕБ СЕРЫЙ ФОРМОВОЙ 400 Г,шт.,0.4,170.53,943,377.2,72631.9
470,214333,107111100,ХЛЕБ СЕРЫЙ ФОРМОВОЙ 250 Г,шт.,0.25,125,1161,290.25,52979.01


In [369]:
df1

,code,wtf,name,unit,weight,cost,sold,sold_in_unit,price
0,34426,101210532,БЕДРО Ц/Б С/ПР КГ,кг.,NaN,975,1.34,1.34,2227.28
1,34431,101210532,КУРИНЫЕ КОСТИ КГ,кг.,NaN,59,134.47,134.47,2392.39
2,34441,101210532,КУРИНЫЙ ФАРШ С/ПР КГ,кг.,NaN,2280,15.09,15.09,34847.4
3,106563,101210532,БЕДРО Ц/БРОЙЛЕРА КУС ВКУС ОХЛ КГ ЛОТОК,кг.,NaN,1583.63,1616.79,1616.79,2558040.1
4,106564,101210532,ГОЛЕНЬ Ц/Б КУС ВКУС ОХЛ КГ ЛОТОК,кг.,NaN,1888.01,1089.13,1089.13,1727337.23
...,...,...,...,...,...,...,...,...,...
467,117391,107111100,ХЛЕБ РЖАНОЙ 400 Г,шт.,0.4,300,141,56.4,16023.71
468,118085,107111100,ХЛЕБ ЛИТОВСКИЙ 400 Г,шт.,0.4,305,304,121.6,37492.32
469,169712,107111100,ХЛЕБ СЕРЫЙ ФОРМОВОЙ 400 Г,шт.,0.4,170.53,943,377.2,72631.9
470,214333,107111100,ХЛЕБ СЕРЫЙ ФОРМОВОЙ 250 Г,шт.,0.25,125,1161,290.25,52979.01


In [370]:
df1[~df1[df1.columns[0]].isin(df_new['code'])]

,code,wtf,name,unit,weight,cost,sold,sold_in_unit,price


In [371]:
df_new[~df_new['code'].isin(df1[df1.columns[0]])]

,code,avg_price,quantity,unit,name,weight,group


In [372]:
df1

,code,wtf,name,unit,weight,cost,sold,sold_in_unit,price
0,34426,101210532,БЕДРО Ц/Б С/ПР КГ,кг.,NaN,975,1.34,1.34,2227.28
1,34431,101210532,КУРИНЫЕ КОСТИ КГ,кг.,NaN,59,134.47,134.47,2392.39
2,34441,101210532,КУРИНЫЙ ФАРШ С/ПР КГ,кг.,NaN,2280,15.09,15.09,34847.4
3,106563,101210532,БЕДРО Ц/БРОЙЛЕРА КУС ВКУС ОХЛ КГ ЛОТОК,кг.,NaN,1583.63,1616.79,1616.79,2558040.1
4,106564,101210532,ГОЛЕНЬ Ц/Б КУС ВКУС ОХЛ КГ ЛОТОК,кг.,NaN,1888.01,1089.13,1089.13,1727337.23
...,...,...,...,...,...,...,...,...,...
467,117391,107111100,ХЛЕБ РЖАНОЙ 400 Г,шт.,0.4,300,141,56.4,16023.71
468,118085,107111100,ХЛЕБ ЛИТОВСКИЙ 400 Г,шт.,0.4,305,304,121.6,37492.32
469,169712,107111100,ХЛЕБ СЕРЫЙ ФОРМОВОЙ 400 Г,шт.,0.4,170.53,943,377.2,72631.9
470,214333,107111100,ХЛЕБ СЕРЫЙ ФОРМОВОЙ 250 Г,шт.,0.25,125,1161,290.25,52979.01


In [373]:
df1.iloc[39]

code                                 191488
wtf                               108511230
name            КОТЛЕТА В ТЕСТЕ ГОВЯЖЬЯ 90Г
unit                                    шт.
weight                                 0.09
cost                                    290
sold                                     19
sold_in_unit                           1.71
price                               1735.92
Name: 39, dtype: object

In [374]:
groups = [
    'Kуры(включая цыплят),  свежий и охл., части тушек не обваленные',
    'Барнина свежая и охл. : четвертины и отруба',
    'Блюда готовые грибные не замор',
    'Блюда готовые из говядины',
    'Блюда готовые из мясо домашней птицы',
    'Блюда готовые овощные не замороженные',
    'Блюда готовые прочие',
    'Блюда готовые рыбные',
    'Вареники',
    'Говядина  свеж. или охлажд.: отруба',
    'Изделия мучные прочие ( блины и т.д ), свежие',
    'Изделия сдобные свежие',
    'Индейки свежие или охлажденные, части тушек не обваленные',
    'Кексы',
    'Конина, включая жеребятину, свежая или охлажденная',
    'Мясо и субпродукты пищ. прочие, свежие,охл. или мор. не включ в др груп',
    'Мясо рыбы без костей прочее, свежее или охлажденное',
    'Пельмени',
    'Печенье сдобное',
    'Пироги, пирожки и пончики свежие',
    'Пирожные свежие',
    'Полуфабрикаты готовые прочие',
    'Полуфабрикаты мясные рубленые, включая фарш и изделия из него',
    'Полуфабрикаты порц. мелкокуск из мяса дом. Птицы',
    'Продукты пищ. из муки, крупы, крахмала, прочие',
    'Рулеты',
    'Свинина свеж. или охл. не обваленная: окорока, лопатки и отруба из них',
    'Смеси для приготовления изделий хлебобулочных, отн. к группам 10.71 и 10.72',
    'Субпродукты и печень птицы дом., свежее и охлажд.',
    'Сухари, гренки и продукты аналогичные обжаренные',
    'Торты свежие',
    'Хлеб пшеничн прочий',
    'Хлеб ржаной'
]

In [375]:
df_new['code'] = df_new['code'].astype('int')
df_new[df_new['group'] == 'Хлеб ржаной'].sort_values(by=['code'])

,code,avg_price,quantity,unit,name,weight,group
346,73148,530.00,6.824,кг.,ХЛЕБ БОЛЬШОЙ ВЕСОВОЙ ЧЕРНЫЙ,1.00,Хлеб ржаной
275,73184,235.88,907.000,шт.,ХЛЕБ КРЕСТЬЯНСКИЙ 400 г,0.40,Хлеб ржаной
450,73197,230.00,66.000,шт.,ХЛЕБ МИНИСТЕРСКИЙ 350 Г,0.35,Хлеб ржаной
451,73213,286.95,287.000,шт.,ХЛЕБ РЖАНОЙ 400 Г,0.40,Хлеб ржаной
453,99571,238.91,740.000,шт.,ХЛЕБ БОРОДИНСКИЙ 400 г,0.40,Хлеб ржаной
254,117391,300.00,141.000,шт.,ХЛЕБ РЖАНОЙ 400 Г,0.40,Хлеб ржаной
295,118085,305.00,304.000,шт.,ХЛЕБ ЛИТОВСКИЙ 400 Г,0.40,Хлеб ржаной
56,169712,170.73,943.000,шт.,ХЛЕБ СЕРЫЙ ФОРМОВОЙ 400 Г,0.40,Хлеб ржаной
76,214333,125.00,1161.000,шт.,ХЛЕБ СЕРЫЙ ФОРМОВОЙ 250 Г,0.25,Хлеб ржаной
371,264422,190.00,119.000,шт.,ХЛЕБ РЖАНОЙ ФОРМОВОЙ 300 Г,0.30,Хлеб ржаной


In [407]:
df_total = pd.DataFrame(columns=['code', 'article', 'name', 'unit', 'weight', 'price', 'sold', 'sold_in_unit', 'acc_price'])

In [408]:
df_total.loc[0] = ['Стат. Отчет по продукции собственного производства', '', '', '', '', '', '', '', '']
df_total.loc[len(df_total)] = ['', '', '', '', '', '', '', '', '']
df_total.loc[len(df_total)] = ['Дата выгрузки', '', '', '', '', '', '', '', '']
df_total.loc[len(df_total)] = ['Площадка', '', '', '', '', '', '', '', '']
df_total.loc[len(df_total)] = ['Пользователь: Робот Ш..', '', '', '', '', '', '', '', '']
df_total.loc[len(df_total)] = ['Альтернативная иерархия: Kуры(включая цыплят),  свежий и охл., части тушек не обваленные [119001],Барнина свежая и охл. : четвертины и отруба [119002],Блюда готовые грибные не замор [119003],Блюда готовые из говядины [119004],Блюда готовые из мясо домашней птицы [119005],Блюда готовые овощные не замороженные [119006],Блюда готовые прочие [119007],Блюда готовые рыбные [119008],Вареники [119038],Говядина  свеж. или охлажд.: отруба [119009],Изделия мучные прочие ( блины и т.д ), свежие [119010],Изделия сдобные свежие [119011],Индейки свежие или охлажденные, части тушек не обваленные [119012],Кексы [119013],Конина, включая жеребятину, свежая или охлажденная [119014],Мясо и субпродукты пищ. прочие, свежие,охл. или мор. не включ в др груп [119015],Мясо рыбы без костей прочее, свежее или охлажденное [119016],Пельмени [119017],Печенье сдобное [119018],Пироги, пирожки и пончики свежие [119019],Пи', '', '', '', '', '', '', '', '']

df_new['code'] = df_new['code'].astype('int')

for group in groups:
    cur_df = df_new[df_new['group'] == group].sort_values(by=['code']).copy()
    cur_df = cur_df.reset_index()
    cur_df = cur_df.drop(['index'], axis=1)
    if len(cur_df) == 0:
        continue
    df_total.loc[len(df_total)] = ['', '', '', '', '', '', '', '', '']
    df_total.loc[len(df_total)] = [f'{group}', '', '', '', '', '', '', '', '']
    df_total.loc[len(df_total)] = ['Код товара', 'Стат. код', 'Наименование', 'Ед. из. умолч.', 'Вес ед. штучного изделия', 'Отпускная цена в ед.изм. умолч.', 'Кол-во реализуемого товара  в ед.изм. умолч.', 'Кол-во реализуемого товара в кг.', 'Сумма по бух. себес.']
    start = len(df_total)
    for row in range(len(cur_df)):
        df_total.loc[len(df_total)] = [cur_df.loc[row, 'code'], '123', cur_df.loc[row, 'name'], cur_df.loc[row, 'unit'], cur_df.loc[row, 'weight'], cur_df.loc[row, 'avg_price'], cur_df.loc[row, 'quantity'], cur_df.loc[row, 'quantity'], '123424221']
    df_total.loc[len(df_total)] = [f'ИТОГО: {group}', '', '', '', '', '', sum(cur_df['quantity']), sum(cur_df['quantity']), sum(df_total.loc[start:, 'acc_price'].astype('float'))]


In [409]:
df_total

,code,article,name,unit,weight,price,sold,sold_in_unit,acc_price
0,Стат. Отчет по продукции собственного производ...,,,,,,,,
1,,,,,,,,,
2,Дата выгрузки,,,,,,,,
3,Площадка,,,,,,,,
4,Пользователь: Робот Ш..,,,,,,,,
...,...,...,...,...,...,...,...,...,...
589,118085,123,ХЛЕБ ЛИТОВСКИЙ 400 Г,шт.,0.4,305.00,304.000,304.000,123424221
590,169712,123,ХЛЕБ СЕРЫЙ ФОРМОВОЙ 400 Г,шт.,0.4,170.73,943.000,943.000,123424221
591,214333,123,ХЛЕБ СЕРЫЙ ФОРМОВОЙ 250 Г,шт.,0.25,125.00,1161.000,1161.000,123424221
592,264422,123,ХЛЕБ РЖАНОЙ ФОРМОВОЙ 300 Г,шт.,0.3,190.00,119.000,119.000,123424221


In [406]:
df_total.to_excel(r'C:\Users\Abdykarim.D\Desktop\kuks.xlsx', header=None, index=False)

In [340]:
sum(df_new['code'])

65259919

In [419]:
drr = df_new.copy()
drr

,code,avg_price,quantity,unit,name,weight,group
0,58712,1977.29,58.612,кг.,САЛАТ ИЗ СВЕКЛЫ С СЫРОМ,1.0,Блюда готовые овощные не замороженные
1,273718,500.00,8.000,шт.,"ШИЕ МОРСЫ/МОРС ИЗ ВИШН,500Г",0.5,Блюда готовые прочие
2,273721,500.00,7.000,шт.,"МОРС ИЗ КЛУБНИКИ ,500Г",0.5,Блюда готовые прочие
3,106568,2056.84,1460.960,кг.,ФИЛЕ Ц/Б КУС ВКУС ОХЛ КГ ЛОТОК,1.0,"Kуры(включая цыплят), свежий и охл., части ту..."
4,58595,3740.75,5.414,кг.,САЛАТ БАКЛАЖАНЫ ОСТРЫЕ,1.0,Блюда готовые овощные не замороженные
...,...,...,...,...,...,...,...
467,69128,4400.00,3.308,кг.,ТОРТ КИЕВСКИЙ,1.0,Торты свежие
468,106567,1210.27,274.527,кг.,ЧЕТВЕРТИНА Ц/Б КУС ВКУС ОХЛ КГ ЛОТОК,1.0,"Kуры(включая цыплят), свежий и охл., части ту..."
469,278181,2800.00,3.490,кг.,"РУЛЕТ АБРИКОСОВЫЙ, КГ",1.0,Рулеты
470,10728,3213.38,3.010,кг.,БРИЗОЛЬ КУРИНАЯ С МОРКОВЧЕЙ,1.0,Блюда готовые из мясо домашней птицы


In [420]:
drr.reset_index(inplace=False)
drr

,code,avg_price,quantity,unit,name,weight,group
0,58712,1977.29,58.612,кг.,САЛАТ ИЗ СВЕКЛЫ С СЫРОМ,1.0,Блюда готовые овощные не замороженные
1,273718,500.00,8.000,шт.,"ШИЕ МОРСЫ/МОРС ИЗ ВИШН,500Г",0.5,Блюда готовые прочие
2,273721,500.00,7.000,шт.,"МОРС ИЗ КЛУБНИКИ ,500Г",0.5,Блюда готовые прочие
3,106568,2056.84,1460.960,кг.,ФИЛЕ Ц/Б КУС ВКУС ОХЛ КГ ЛОТОК,1.0,"Kуры(включая цыплят), свежий и охл., части ту..."
4,58595,3740.75,5.414,кг.,САЛАТ БАКЛАЖАНЫ ОСТРЫЕ,1.0,Блюда готовые овощные не замороженные
...,...,...,...,...,...,...,...
467,69128,4400.00,3.308,кг.,ТОРТ КИЕВСКИЙ,1.0,Торты свежие
468,106567,1210.27,274.527,кг.,ЧЕТВЕРТИНА Ц/Б КУС ВКУС ОХЛ КГ ЛОТОК,1.0,"Kуры(включая цыплят), свежий и охл., части ту..."
469,278181,2800.00,3.490,кг.,"РУЛЕТ АБРИКОСОВЫЙ, КГ",1.0,Рулеты
470,10728,3213.38,3.010,кг.,БРИЗОЛЬ КУРИНАЯ С МОРКОВЧЕЙ,1.0,Блюда готовые из мясо домашней птицы


In [217]:
df2 = pd.read_excel(r'C:\Users\Abdykarim.D\Desktop\kokl.xlsx')
df2 = df2.dropna(how='all')
df2 = df2[df2[df2.columns[0]] != 'Код товара']
df2 = df2.dropna(subset=[df2.columns[2]])
df2

,Стат. Отчет по продукции собственного производства,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
8,10732,108511230,БРИЗОЛЬ ГОВЯЖЬЯ С ОГУРЦАМИ,кг.,NaN,5219.62,2.51,2.51,5106.81
9,15832,108511230,ГОЛУБЦЫ С МЯСОМ(ГОВ)И РИСОМ,кг.,NaN,2407.86,0.91,0.91,898.46
10,30968,108511230,КОТЛЕТЫ МОСКОВС ЖАРЕНЫЕ,кг.,NaN,5347.66,2.93,2.93,6699.09
11,41233,108511230,МЯСО ПО-ТАЙСКИ,кг.,NaN,5922.45,12.26,12.26,30403.84
12,41235,108511230,МЯСО ПО-ФРАНЦУЗСКИ,кг.,NaN,4459.12,0.83,0.83,1469.23
13,53093,108511230,ПЛОВ ИЗ ГОВЯДИНЫ,кг.,NaN,3223.99,30.97,30.97,38523.32
14,53906,108511230,ПОДЖАРКА ИЗ ГОВЯДИНЫ,кг.,NaN,10080,0.24,0.24,1248.97
15,58727,108511230,САЛАТ КАВКАЗ МЯСНОЙ,кг.,NaN,4590,0.94,0.94,1913.85
16,58829,108511230,САЛАТ МЯСНОЙ,кг.,NaN,3791.25,9.24,9.24,12790.89
17,58842,108511230,САЛАТ ОГУРЦЫ С МЯСОМ,кг.,NaN,4313.99,42.61,42.61,97093.4


In [129]:
df_new['group'].unique()

array(['Kуры(включая цыплят),  свежий и охл., части тушек не обваленные',
       'Барнина свежая и охл. : четвертины и отруба',
       'Блюда готовые грибные не замор', 'Блюда готовые из говядины'],
      dtype=object)

In [ ]:
import psycopg2
import csv
import pandas as pd

start_date = '2023-01-01'
end_date = '2023-03-31'
store_id = '985'

conn = psycopg2.connect(dbname='adb', host='172.16.10.22', port='5432',
                        user='rpa_robot', password='Qaz123123+')

cur = conn.cursor(name='1583_first_part')

query = f"""
    select source_product_id, sum(quantity) FROM "dwh_data"."qs_fact_sales" q where source_product_id in ('10732', '15832', '30968', '41233', '41235', '53093', '53906', '58727', '58829', '58842', '58918', '58928', '68704', '69282', '116694', '124383', '174784', '191488', '212419', '284822') and source_store_id = 116 and date(order_date) between to_date('{start_date}', 'YYYY-MM-DD') and to_date('{end_date}', 'YYYY-MM-DD')
    group by source_product_id;
"""


cur.execute(query)

print('Executed')

df3 = pd.DataFrame(cur.fetchall())

cur.close()
conn.close()

In [133]:
df3

,0,1
0,68704,1.374
1,212419,3.000
2,53093,30.970
3,58928,13.650
4,41235,0.828
5,69282,6.106
6,58829,9.240
7,58727,0.938
8,15832,0.906
9,10732,2.514


In [139]:
a = ['15799', '30694', '34421', '34423', '34426', '34427', '34431', '34432', '34433', '34434', '34435', '34439', '34441', '34443', '35207', '48284', '48706', '71322', '73600', '106563', '106564', '106565', '106566', '106567', '106568', '106572', '106699', '109309', '109310', '109311', '109312', '109314', '109317', '111062', '122917', '122919', '122920', '122924', '123321', '123322', '123325', '123326', '123327', '130737', '130739', '130740', '130741', '130742', '130743', '147761', '147762', '147763', '147764', '147765', '147766', '147767', '147768', '151021', '175464', '175470', '175473', '175475', '175480', '176650', '190811', '190812', '193472', '193473', '193474', '193475', '193476', '193478', '193479', '193480', '193481', '193484', '193487', '193488', '193489', '193490', '193492', '193493', '198679', '198680', '198681', '198682', '198683', '198684', '198688', '203379', '205522', '206026', '206027', '206028', '206029', '206030', '206031', '206038', '206255', '206256', '206350', '206351', '206352', '210968', '210970', '213129', '213133', '213134', '213135', '213136', '213137', '213138', '219749', '219750', '219751', '219752', '219753', '219756', '226766', '226767', '226768', '226769', '226770', '226771', '227317', '227318', '227319', '227320', '227321', '233399', '233401', '234646', '234647', '234653', '234666', '234689', '234690', '234693', '240314', '240316', '240317', '242447', '242449', '242451', '245120', '247451', '247482', '247483', '250527', '250528', '250529', '250530', '250531', '250532', '251430', '251431', '252241', '252941', '257337', '257338', '263169', '267398', '267538', '272212', '272213', '272214', '272215', '272216', '272217', '275333', '290672', '295432', '295436', '299183', '299185', '9515', '9519', '9524', '9525', '9528', '9529', '9530', '9543', '9551', '48047', '48048', '48050', '48051', '48491', '78059', '142555', '145640', '170319', '175462', '193458', '193459', '193460', '193461', '193462', '193463', '193464', '193465', '193466', '210969', '241080', '256601', '256602', '258397', '258398', '258399', '258400', '258800', '258801', '258802', '258803', '258872', '259019', '259020', '259021', '259023', '262733', '263014', '263015', '263016', '263017', '263018', '263019', '263020', '263021', '263022', '263023', '263024', '263025', '263026', '263027', '263028', '263029', '263030', '263031', '263032', '263033', '263034', '263035', '263036', '263037', '263038', '263039', '263041', '263115', '264584', '266224', '272207', '272208', '275894', '278473', '287210', '298501', '16508', '19408', '47295', '58632', '58634', '58635', '58657', '58670', '58671', '58693', '58718', '58732', '58758', '58872', '58914', '90158', '96938', '117945', '136163', '150350', '206864', '210342', '214115', '230310', '230311', '235783', '235925', '257315', '264452', '264739', '266744', '267607', '270457', '270458', '270473', '271625', '271673', '279741', '7039', '8472', '8851', '9914', '9916', '9917', '10077', '10079', '10080', '10081', '10105', '10731', '10732', '14852', '14853', '14935', '15723', '15732', '15832', '16912', '18087', '18088', '19035', '27323', '30968', '30969', '30971', '30980', '31001', '34589', '34595', '41232', '41233', '41235', '50733', '53093', '53906', '58583', '58593', '58626', '58628', '58672', '58695', '58696', '58727', '58743', '58777', '58783', '58828', '58829', '58833', '58842', '58889', '58891', '58905', '58918', '58919', '58923', '58928', '58931', '58947', '68704', '69282', '70223', '79047', '96939', '116694', '116696', '118566', '124383', '138299', '139092', '139094', '150373', '152155', '152165', '152167', '152180', '159886', '167149', '174784', '186008', '190250', '191488', '191490', '205661', '206865', '210186', '210259', '210344', '210386', '212419', '233594', '234055', '237087', '251748', '257317', '264360', '264442', '264443', '264444', '264446', '264447', '264448', '264451', '264453', '264454', '264455', '264456', '264457', '264458', '264459', '264460', '264463', '264466', '264721', '264722', '264723', '264724', '264736', '264738', '264741', '264742', '264745', '264746', '266661', '266671', '266676', '266677', '266679', '266686', '266690', '266691', '266693', '266695', '266696', '266752', '266753', '267595', '267597', '267812', '268042', '268044', '268243', '268529', '268949', '268988', '269086', '269571', '269621', '269635', '271604', '271605', '271606', '271633', '271640', '271650', '271654', '271667', '273360', '274189', '274406', '274835', '278055', '278646', '280538', '280540', '280542', '284822', '288141', '301184', '301220', '307569', '309161', '309293', '309295', '309301', '313746', '314032', '314033', '6244', '9836', '10084', '10087', '10090', '10094', '10104', '10727', '10728', '10729', '15790', '15791', '15795', '16501', '18089', '18090', '18373', '27332', '30937', '30957', '30976', '30977', '30979', '30999', '32800', '33435', '33440', '34425', '34451', '34455', '34458', '34459', '34545', '34547', '34548', '34596', '46379', '47116', '47124', '47146', '53095', '57665', '58033', '58036', '58039', '58041', '58096', '58097', '58098', '58099', '58100', '58102', '58585', '58586', '58625', '58753', '58775', '58776', '58846', '58874', '58894', '58895', '58925', '58948', '70224', '71312', '71315', '71316', '71321', '71328', '73615', '74406', '75937', '85055', '86247', '101096', '101097', '116691', '116692', '116693', '116794', '119370', '120009', '137873', '137875', '139093', '149695', '150347', '151680', '151681', '151682', '152130', '152152', '152154', '152156', '152157', '152158', '152159', '152160', '152162', '152166', '152168', '152169', '152170', '152171', '152172', '152173', '152174', '152177', '152178', '152181', '157769', '160544', '162301', '162327', '168522', '169072', '177612', '185545', '187461', '190246', '190249', '191487', '191489', '206925', '207871', '210184', '210185', '210258', '210322', '210385', '210387', '210638', '210639', '210640', '210641', '210986', '211128', '211132', '211680', '212415', '212752', '214114', '230304', '233360', '233832', '233833', '234057', '234375', '235276', '235782', '235785', '235786', '237078', '237083', '237084', '237086', '237178', '249096', '257325', '264441', '264461', '264468', '264469', '264470', '264471', '264472', '264473', '264474', '264475', '264476', '264477', '264478', '264479', '264480', '264481', '264482', '264483', '264484', '266754', '266755', '267586', '267594', '268043', '268244', '268748', '268751', '268752', '268950', '269084', '269085', '269622', '269626', '270405', '271617', '271623', '271624', '271626', '271629', '271631', '271632', '271634', '271637', '271638', '271639', '271641', '271644', '271649', '271652', '271653', '271659', '271665', '271670', '271672', '273031', '274188', '274407', '274836', '275155', '278083', '278085', '278088', '278090', '278092', '278094', '278479', '278645', '280551', '284400', '284592', '284597', '284600', '284605', '284821', '285486', '286109', '287433', '292087', '296563', '296564', '296914', '309601', '313796', '314034', '314038', '9010', '18372', '19750', '22037', '22058', '22693', '22943', '22970', '22979', '22980', '22989', '22992', '22996', '22999', '23001', '23466', '23470', '23471', '23484', '23488', '23489', '23491', '23492', '23493', '23500', '23514', '34290', '34292', '35783', '36361', '40042', '40046', '43651', '43652', '46919', '46973', '46998', '47134', '48199', '48208', '54740', '54766', '54772', '56696', '56878', '58578', '58587', '58592', '58595', '58596', '58598', '58599', '58600', '58601', '58604', '58614', '58617', '58619', '58630', '58643', '58644', '58663', '58667', '58677', '58703', '58707', '58712', '58717', '58726', '58734', '58735', '58737', '58740', '58749', '58754', '58763', '58780', '58784', '58801', '58825', '58837', '58857', '58866', '60212', '60215', '63211', '63419', '63619', '63687', '84778', '84793', '85651', '86236', '86237', '86240', '86244', '86246', '88127', '88129', '96792', '97349', '98240', '107421', '118560', '118890', '119010', '121179', '121180', '121362', '123619', '123947', '127086', '130354', '138343', '138344', '138779', '150349', '150434', '150442', '150443', '150477', '166880', '170666', '173980', '184578', '184579', '184580', '184838', '210182', '210183', '210326', '210734', '210908', '210912', '210914', '210916', '210918', '210919', '211043', '211208', '211662', '213661', '214460', '214461', '217166', '230307', '230308', '230312', '234413', '234415', '235781', '235784', '237076', '237832', '237933', '240453', '258616', '263886', '263887', '264754', '264756', '264757', '267599', '268190', '268191', '268193', '268633', '268737', '268740', '268742', '268744', '268746', '268951', '269511', '269512', '269965', '270461', '270471', '270474', '271612', '271615', '271616', '271621', '271635', '271646', '271647', '271656', '271657', '271662', '271664', '271666', '271674', '271675', '273217', '273218', '273219', '273792', '273793', '274153', '274154', '274155', '274156', '274849', '274850', '274851', '274852', '274999', '275157', '275401', '275652', '275655', '275662', '275663', '275841', '275842', '275843', '275844', '278053', '278082', '278084', '278086', '278087', '278089', '278091', '278236', '279450', '279452', '279456', '279459', '279460', '279740', '279742', '279743', '279744', '279748', '280547', '280548', '280549', '280550', '280552', '284819', '284994', '284995', '285491', '288712', '301219', '301221', '301222', '301223', '301224', '301225', '301226', '301227', '301228', '301229', '301231', '301232', '301233', '301234', '301235', '301236', '301239', '301242', '301258', '301259', '301260', '301262', '301266', '301267', '301269', '301270', '301272', '301273', '301274', '301276', '301278', '301280', '308515', '308527', '309306', '309487', '309489', '309490', '8850', '8858', '9532', '9536', '9538', '9547', '9900', '14928', '14932', '16016', '16293', '18091', '18092', '18374', '19047', '19830', '19837', '20185', '22700', '22720', '22973', '27312', '27313', '27339', '28293', '30143', '30152', '30984', '31190', '31195', '31196', '31198', '31200', '31206', '34593', '35607', '36332', '36800', '43748', '43749', '43750', '43751', '46908', '47085', '47133', '47136', '47137', '47138', '47201', '47214', '48439', '50943', '50963', '51655', '52638', '52640', '52647', '53089', '53090', '57587', '57594', '58093', '58579', '58591', '58609', '58616', '58618', '58623', '58631', '58638', '58665', '58678', '58684', '58689', '58690', '58694', '58699', '58705', '58706', '58708', '58709', '58710', '58713', '58719', '58720', '58731', '58733', '58761', '58779', '58802', '58803', '58835', '58838', '58839', '58843', '58844', '58850', '58881', '58893', '58912', '58921', '58922', '58939', '58941', '59904', '59906', '59907', '59908', '59909', '59910', '59911', '59912', '59915', '59917', '59918', '59919', '59922', '59923', '63860', '63881', '63882', '64090', '64103', '64104', '64998', '65001', '65004', '65009', '65291', '66851', '66853', '66854', '69259', '70760', '70932', '70933', '71329', '73086', '73481', '74541', '74717', '74904', '74905', '74908', '75925', '75932', '75933', '77342', '77706', '77718', '77719', '77784', '79044', '85056', '85543', '85548', '85551', '86241', '87073', '88098', '88101', '88131', '88134', '90084', '90198', '90241', '90548', '96937', '97686', '100047', '101098', '109797', '109798', '116685', '116687', '116688', '116689', '116690', '116717', '118462', '119008', '119064', '120244', '120426', '120427', '121663', '123152', '123580', '125115', '127071', '131381', '134094', '134675', '134887', '135100', '135265', '136057', '138300', '138301', '138777', '138778', '139095', '145013', '146859', '150351', '151683', '151684', '152161', '152163', '154955', '157421', '158544', '158869', '160170', '164006', '166123', '166125', '166135', '166151', '166152', '166153', '167155', '167579', '168009', '168189', '169050', '170667', '170668', '170669', '170670', '170711', '173654', '173962', '176257', '176743', '184576', '184836', '184837', '184910', '186007', '188421', '190245', '191603', '192789', '194663', '194744', '205662', '206866', '210257', '210915', '210917', '211209', '212414', '212416', '212417', '212418', '213662', '214112', '214116', '214117', '219743', '219748', '226179', '226180', '226181', '226182', '226183', '226184', '226185', '226348', '226699', '226701', '226709', '226895', '226899', '227092', '229135', '230313', '235288', '237079', '237080', '237081', '237082', '238703', '238704', '239041', '241798', '243234', '247115', '248039', '249441', '250483', '252800', '252805', '252806', '252807', '252915', '253387', '254903', '254904', '254905', '254906', '254907', '254908', '254912', '254913', '254914', '255097', '255543', '255544', '255545', '255546', '255631', '255685', '255687', '255688', '255691', '255693', '255695', '255696', '255698', '255703', '255705', '255706', '255711', '255713', '255717', '255745', '256645', '256646', '257318', '257319', '257329', '257454', '257651', '257686', '259156', '260505', '260508', '260509', '260510', '260511', '260515', '260516', '260517', '260519', '260521', '260522', '260525', '260540', '261664', '261665', '261667', '261668', '263984', '263985', '263986', '264164', '264165', '264167', '264343', '264345', '264346', '264347', '264357', '264363', '264365', '264367', '264368', '264369', '264371', '264373', '264374', '264375', '264376', '264380', '264381', '264382', '264383', '264384', '264386', '264387', '264434', '264435', '264436', '264437', '264438', '264439', '264440', '264445', '264449', '264450', '264743', '264744', '264748', '264755', '264758', '265070', '265798', '265799', '265800', '265801', '265802', '265808', '266013', '266673', '266678', '266681', '266689', '266692', '266694', '266745', '266747', '266756', '266757', '267420', '267423', '267424', '267426', '267527', '267587', '267598', '267601', '267637', '267811', '267813', '267838', '267947', '267948', '267953', '267954', '267955', '267956', '268171', '268172', '268173', '268174', '268175', '268176', '268177', '268178', '268179', '268180', '268181', '268186', '268187', '268188', '268189', '268192', '268221', '268223', '268238', '268240', '268242', '268260', '268261', '268358', '268359', '268360', '268414', '268480', '268481', '268532', '268533', '268534', '268535', '268632', '268635', '268666', '268681', '268747', '268749', '268750', '268754', '268830', '268856', '268857', '268859', '268860', '268984', '269627', '269707', '269708', '269968', '269970', '270072', '270470', '270475', '270670', '270674', '270682', '270814', '270815', '270816', '270817', '270818', '270819', '270821', '270822', '270823', '270824', '270825', '270890', '270914', '270916', '270918', '270919', '270922', '270923', '270924', '270925', '271145', '271603', '271619', '271630', '271636', '271645', '271648', '271658', '271663', '271668', '271671', '272259', '272275', '272443', '272445', '272446', '272447', '272594', '272597', '272601', '272794', '272797', '273094', '273095', '273096', '273361', '273362', '273363', '273364', '273403', '273404', '273405', '273541', '273542', '273543', '273544', '273646', '273647', '273648', '273649', '273650', '273651', '273710', '273711', '273712', '273713', '273714', '273715', '273716', '273717', '273718', '273719', '273720', '273721', '273821', '273878', '273882', '273884', '273890', '273896', '273902', '273930', '273944', '274040', '274400', '274401', '274402', '274403', '274404', '274405', '275000', '275653', '275695', '276505', '276584', '276589', '276595', '276601', '277027', '277030', '277031', '277032', '277226', '277227', '277228', '277229', '277240', '277241', '277242', '277244', '277245', '277246', '277306', '277907', '278093', '278647', '279389', '280465', '280466', '280467', '280468', '280537', '280539', '281450', '281451', '281547', '282349', '282360', '282538', '283676', '284359', '284367', '284370', '284393', '284823', '285293', '285359', '285360', '285361', '285362', '285367', '285372', '285374', '285382', '285383', '285384', '285485', '285487', '285488', '285865', '285880', '285882', '286005', '286006', '286008', '286016', '286019', '286020', '286021', '286022', '286026', '286094', '286100', '286102', '288071', '288073', '288075', '288076', '288116', '288124', '288125', '288126', '288128', '288130', '288131', '288132', '288133', '288134', '288136', '288138', '288215', '288218', '288219', '288220', '288221', '288223', '288267', '288637', '288639', '288641', '288643', '288645', '288649', '288814', '288815', '289053', '289054', '289069', '289070', '289178', '289356', '289360', '289363', '289365', '289855', '290192', '290193', '290194', '290218', '290219', '290220', '290676', '290684', '290776', '290907', '290908', '290916', '290917', '290918', '290919', '290921', '290927', '290929', '290930', '290933', '290935', '290937', '290938', '290939', '290940', '291298', '292025', '292027', '292062', '292063', '292064', '292065', '292066', '292067', '292068', '292070', '292076', '292085', '292086', '292126', '292132', '293846', '293847', '294454', '294455', '294456', '294458', '294459', '294460', '294461', '294462', '294463', '294636', '294871', '294898', '294901', '294942', '294943', '294944', '295765', '295766', '296257', '296258', '296261', '296453', '296637', '296653', '296711', '296714', '296716', '296718', '296719', '296720', '296721', '296722', '296724', '296728', '296729', '296730', '296775', '296797', '296798', '296800', '296801', '296804', '296807', '296809', '296810', '296813', '296937', '296941', '296942', '296944', '296946', '296994', '296995', '296996', '296997', '296998', '296999', '297000', '297001', '297066', '297070', '297316', '297608', '297612', '297619', '297736', '300805', '300806', '300807', '300808', '300809', '300810', '300811', '300812', '300813', '300814', '300815', '300833', '300834', '300858', '300862', '301263', '301268', '302329', '302709', '302710', '302711', '302712', '302713', '302714', '302715', '303629', '304854', '304855', '305044', '305045', '305046', '305048', '305049', '305053', '305054', '305055', '305554', '305799', '305857', '305898', '306742', '306743', '307525', '307527', '307725', '307727', '307729', '307791', '307792', '307799', '307808', '307809', '307877', '307878', '307879', '307880', '307881', '308002', '308006', '308009', '308015', '308017', '308018', '308022', '308045', '308062', '308065', '308068', '308069', '308071', '308076', '308079', '308242', '308471', '308472', '308473', '308474', '308475', '308476', '308534', '308583', '308584', '308866', '308922', '308924', '308925', '308926', '308928', '308930', '308931', '308947', '309122', '309125', '309132', '309144', '309145', '309146', '309148', '309149', '309166', '309167', '309296', '309299', '309302', '309303', '309307', '309309', '309310', '309311', '309316', '309320', '309321', '309332', '309343', '309344', '309346', '309347', '309356', '309357', '309358', '309366', '309368', '309369', '309371', '309372', '309375', '309377', '309380', '309384', '309385', '309409', '309410', '309414', '309415', '309424', '309468', '309561', '309562', '309563', '309564', '309565', '309566', '309568', '309570', '309572', '309591', '309592', '309593', '309595', '309596', '309602', '310260', '310327', '310974', '313754', '313788', '313792', '313794', '313798', '313799', '313801', '313804', '313805', '313806', '313807', '313808', '313809', '313811', '313812', '313814', '313815', '313818', '313819', '313822', '313824', '313832', '313833', '313836', '313841', '313844', '313846', '313849', '313853', '313857', '314085', '314157', '322607', '322608', '322678', '323004', '323005', '323006', '323007', '323008', '323010', '323015', '323018', '323019', '323020', '324233', '330505', '330506', '331682', '8862', '10082', '10096', '18353', '19751', '22873', '22874', '23305', '23306', '23307', '30991', '39655', '39656', '39665', '45918', '48745', '52328', '56970', '58095', '58376', '58534', '58535', '58655', '58656', '58661', '58673', '58674', '58714', '58827', '58884', '58917', '58926', '58927', '58932', '58934', '60840', '61070', '65002', '65659', '65662', '67965', '70236', '70242', '71303', '71340', '71341', '71344', '71364', '71365', '71366', '71373', '71374', '71392', '71394', '71395', '71397', '71404', '71405', '71406', '71407', '71408', '71409', '71413', '71415', '71416', '71433', '71444', '71949', '71967', '75934', '84738', '85396', '99705', '115096', '117515', '117516', '118373', '120224', '122963', '129817', '130025', '130026', '130027', '130029', '130030', '130031', '130160', '130317', '130353', '130677', '130678', '130680', '130732', '139091', '150945', '152042', '152164', '152175', '152176', '152179', '152203', '152576', '171321', '171356', '171358', '171359', '171373', '173879', '175656', '177614', '183160', '184243', '185411', '191110', '191112', '198128', '198129', '198258', '199931', '199932', '200155', '200212', '200214', '200215', '200326', '200327', '200328', '200329', '200330', '200331', '200332', '200387', '200388', '200389', '200392', '200393', '200483', '200484', '200485', '212753', '213995', '230301', '237085', '247879', '248040', '264467', '264750', '264751', '264752', '264753', '266009', '266011', '266012', '266748', '266750', '266751', '267589', '267596', '267600', '268185', '268738', '268739', '268743', '268745', '270641', '271642', '271655', '271660', '271661', '272888', '272889', '272890', '273136', '273138', '276808', '279746', '282912', '285869', '285870', '287788', '287793', '287794', '287795', '287853', '290197', '290198', '296561', '296562', '297615', '313754', '313898', '15666', '15670', '15671', '15684', '15685', '15686', '15690', '15692', '15700', '15703', '15704', '15705', '15707', '15712', '15713', '15714', '15716', '15724', '15725', '15730', '15734', '15739', '15740', '15741', '15742', '15779', '48336', '105609', '118521', '156970', '169330', '169644', '175450', '175453', '175454', '175457', '175461', '193417', '193419', '193420', '193421', '193422', '193423', '193444', '193445', '193446', '193447', '193448', '193450', '193451', '193452', '193453', '194338', '194339', '194340', '195059', '195060', '196244', '203291', '210965', '210966', '210967', '228969', '253083', '253186', '253187', '253188', '253190', '258376', '258377', '258379', '258380', '258381', '258382', '258383', '258384', '258778', '258779', '258780', '258781', '258782', '258783', '258784', '258785', '258786', '258804', '258805', '258871', '262731', '263042', '263043', '263044', '263045', '263046', '263047', '263048', '263049', '263050', '263051', '263052', '263053', '263054', '263055', '263056', '263057', '263058', '263059', '263060', '263061', '263062', '263063', '263066', '263067', '263068', '263069', '263070', '263071', '263072', '263073', '263074', '263075', '263076', '263077', '263081', '263082', '263083', '263084', '263085', '263086', '263087', '263088', '263089', '263090', '263091', '263092', '263093', '263094', '263095', '263096', '263097', '263098', '263100', '263102', '263103', '263104', '263105', '263114', '263116', '263163', '263953', '266226', '266227', '269831', '271613', '271614', '272197', '272198', '272199', '272200', '272201', '272202', '272204', '272205', '272435', '272436', '272437', '272438', '272661', '272663', '272664', '272665', '273692', '277809', '298500', '299188', '9821', '10172', '10186', '10188', '10190', '10193', '10195', '19572', '19573', '19574', '19576', '35599', '35611', '47147', '59927', '68995', '74335', '84641', '96793', '96794', '96795', '96796', '120465', '123049', '130091', '148914', '179284', '179285', '179286', '179863', '181457', '185064', '185066', '220332', '226707', '226708', '264372', '264785', '267823', '268369', '268636', '268649', '271620', '271622', '271651', '281947', '281990', '296803', '296805', '296806', '296808', '296947', '296949', '301279', '309124', '320932', '320935', '320939', '320941', '321232', '9272', '10857', '10859', '10863', '10896', '10897', '10908', '10910', '10916', '10917', '10921', '10922', '10924', '10926', '10930', '10932', '10936', '10938', '10939', '10941', '10942', '10945', '10946', '10948', '10950', '10952', '10956', '10958', '10961', '10963', '10965', '10966', '10972', '10975', '10977', '10980', '10982', '10983', '10985', '10987', '10988', '10989', '10992', '10997', '11670', '11848', '11850', '11855', '11857', '11858', '17357', '22754', '23004', '28351', '32851', '32864', '32865', '32866', '32867', '34351', '35612', '48131', '48247', '48524', '48656', '49406', '53042', '53138', '53139', '53140', '53142', '53145', '53146', '57667', '57668', '57823', '60784', '60785', '60786', '60789', '62365', '62376', '62392', '62393', '62395', '72611', '72612', '76766', '76769', '76771', '76772', '79482', '84325', '84334', '84335', '85121', '85122', '85185', '85186', '85187', '87226', '88100', '116681', '116682', '116684', '128965', '132974', '133139', '134085', '134247', '137582', '138136', '138432', '149291', '151564', '151597', '154910', '154978', '154979', '156906', '165363', '169120', '169121', '170653', '173867', '174847', '184982', '184983', '184984', '186392', '188588', '195284', '198281', '198283', '199026', '200403', '202320', '202321', '204421', '210656', '210733', '212189', '213450', '234746', '247315', '247316', '247317', '247318', '247319', '248285', '257033', '257034', '257035', '257037', '257039', '257040', '257041', '257091', '263944', '263945', '263947', '263948', '264196', '264197', '267209', '267211', '267212', '267214', '269636', '270715', '270716', '270717', '271628', '272240', '272248', '272249', '272250', '272251', '272252', '272253', '272254', '272735', '272741', '272743', '274031', '274032', '275002', '277808', '277846', '277847', '277848', '277849', '277850', '277946', '278246', '278253', '279310', '279311', '279312', '279313', '279314', '279315', '279316', '281217', '281223', '282634', '282649', '283024', '283025', '283026', '283027', '283946', '284075', '284078', '284081', '284356', '285014', '285015', '285016', '285017', '285019', '285385', '285386', '285387', '285388', '285389', '287957', '287959', '287960', '291072', '291082', '291084', '291116', '291117', '291119', '291120', '291121', '291441', '291442', '291656', '291657', '291658', '291659', '292113', '292114', '292115', '294679', '296666', '297611', '309604', '309606', '309607', '309608', '309611', '309612', '309613', '309614', '313848', '313852', '313855', '15785', '19118', '50947', '61301', '175466', '175468', '175474', '175477', '175494', '175497', '193471', '193482', '193485', '193491', '282278', '282279', '282805', '282806', '282807', '282808', '282809', '24297', '24300', '24301', '24309', '24321', '24322', '24334', '24340', '52328', '124015', '124016', '124017', '134674', '176094', '252292', '256301', '256302', '256303', '256304', '260578', '263992', '267364', '267365', '267366', '267377', '272237', '272238', '278208', '282544', '282545', '298981', '298982', '28502', '28507', '28530', '115085', '170318', '191062', '193469', '193470', '234376', '255037', '255093', '256476', '263106', '263107', '263108', '263109', '263110', '263111', '263113', '263954', '267967', '268537', '269947', '272210', '272211', '280390', '9534', '15678', '15679', '15694', '15695', '15696', '15697', '15701', '15715', '15721', '15727', '30134', '35428', '60684', '60685', '60686', '60687', '60706', '60729', '60742', '61295', '77700', '106148', '152074', '152075', '152076', '152077', '162850', '162851', '162852', '162853', '162854', '175498', '193449', '193456', '224430', '226589', '232958', '261356', '262296', '263064', '263065', '263078', '263079', '263080', '263112', '263131', '263142', '263150', '263157', '263158', '263170', '263171', '263172', '264324', '264591', '293189', '293190', '293191', '293192', '293193', '293194', '293195', '293196', '293197', '293198', '293199', '293200', '293201', '293202', '302666', '302667', '302668', '302669', '302670', '302671', '302672', '302673', '302674', '302675', '305512', '309305', '9909', '22872', '22880', '23308', '23365', '26432', '39186', '39666', '42910', '42912', '42913', '42914', '42915', '42916', '48113', '48187', '48194', '48195', '48196', '48206', '48207', '48280', '48328', '48343', '48344', '48345', '48387', '48493', '48494', '48495', '48496', '48504', '48511', '48512', '48513', '48514', '48516', '48517', '48527', '48585', '48657', '48659', '48660', '48661', '48662', '48663', '48664', '48665', '48667', '48701', '48718', '48721', '48723', '48724', '48733', '48734', '48737', '48738', '48739', '48740', '48741', '48742', '48746', '48747', '48748', '48749', '48750', '48751', '48752', '48754', '48755', '48756', '48758', '48759', '48760', '48761', '56971', '56973', '58288', '58289', '58290', '58308', '58313', '58314', '58328', '58336', '58339', '58340', '58343', '58352', '58353', '58533', '58538', '58540', '60897', '61132', '63835', '63839', '65647', '65654', '65658', '71347', '71402', '71427', '71945', '71977', '77209', '78852', '79040', '85724', '85725', '85788', '87286', '89875', '99823', '99824', '106933', '112730', '113194', '113195', '113969', '114278', '115481', '116756', '117721', '127087', '127088', '127103', '129819', '130340', '130674', '130692', '130693', '130694', '130695', '130696', '130697', '130698', '130699', '130700', '130728', '130756', '136090', '136091', '137833', '144590', '144591', '146858', '147489', '149843', '151136', '151750', '151751', '151752', '151753', '151778', '151779', '151780', '151781', '151861', '152043', '152057', '152058', '152059', '152060', '152182', '152183', '152232', '152577', '157679', '157680', '157681', '157682', '157683', '157684', '157785', '157786', '157787', '157788', '157823', '157824', '157825', '157826', '159749', '164637', '165409', '166911', '167485', '171319', '171320', '173619', '173888', '173893', '175657', '176729', '178420', '178424', '178699', '178917', '178920', '179487', '180644', '182130', '183158', '184330', '184331', '184333', '184363', '185463', '186075', '186944', '187961', '188072', '188074', '188075', '188645', '192187', '192480', '192785', '196197', '196198', '196199', '196234', '196235', '196236', '196237', '196238', '196243', '196245', '196246', '196247', '196266', '196267', '196268', '196269', '196270', '196271', '196273', '196274', '196275', '196460', '196461', '196688', '196689', '196972', '196974', '197055', '197202', '197203', '197204', '197205', '197206', '197207', '197208', '197376', '197460', '197755', '197757', '197758', '197760', '198126', '198131', '198132', '198147', '198367', '198458', '198459', '198460', '198914', '199184', '199185', '199268', '199270', '199935', '199936', '200201', '200203', '200205', '200208', '200210', '200211', '200223', '200415', '200416', '200417', '200418', '200550', '202210', '202276', '202277', '202280', '224596', '224602', '224605', '224607', '224608', '224610', '224611', '224612', '224613', '224615', '225931', '235323', '247491', '247551', '247553', '247880', '247881', '247883', '251491', '251492', '254853', '254855', '254858', '254859', '262639', '264815', '266232', '267532', '267533', '267547', '267554', '267620', '267660', '273125', '279443', '284158', '290206', '290222', '293848', '293849', '296506', '299187', '304696', '305024', '313757', '313861', '313891', '319888', '320018', '320074', '19571', '19575', '37149', '37151', '37158', '50198', '50207', '50208', '50210', '50211', '50212', '52072', '85932', '135489', '164485', '164486', '164487', '164489', '164491', '164492', '164500', '164568', '164571', '164605', '164606', '190230', '227199', '229134', '264462', '264465', '265685', '267809', '271627', '307568', '307570', '309162', '28001', '51211', '51223', '51224', '51276', '51281', '51311', '51327', '51328', '51337', '51339', '51351', '51359', '51412', '51414', '51416', '51417', '51420', '51425', '51430', '51431', '51434', '51435', '51440', '51469', '51470', '51475', '51483', '51492', '51514', '51552', '51560', '51608', '51610', '51620', '51651', '51652', '51662', '51666', '73082', '74324', '74329', '74331', '74332', '74953', '85909', '85911', '85916', '88121', '97348', '98236', '118252', '118253', '118254', '119033', '119034', '119035', '134093', '134098', '134099', '161318', '169473', '169474', '170628', '173897', '176093', '180267', '181792', '194290', '200007', '200722', '202391', '202396', '257260', '257262', '257263', '257343', '257344', '257924', '259687', '259688', '267293', '267294', '267295', '267296', '267298', '267299', '267498', '267505', '267506', '267515', '267516', '267517', '267518', '267558', '267559', '267560', '267561', '267562', '267563', '267564', '267565', '267618', '267619', '268052', '268644', '268645', '268646', '268647', '268648', '269034', '269934', '269935', '269936', '269938', '271846', '271939', '272122', '272123', '272124', '272125', '272126', '272128', '272129', '272130', '272131', '272134', '272135', '272136', '272137', '272139', '272140', '272141', '272142', '273864', '278209', '278210', '282541', '282542', '282680', '283951', '285634', '291994', '302394', '307310', '307456', '307457', '307458', '307459', '307468', '307469', '307470', '307471', '52251', '52258', '52259', '52263', '52264', '52267', '52268', '52279', '52280', '52282', '52284', '52285', '52286', '52289', '52290', '52293', '52294', '52297', '52305', '52309', '52310', '52311', '52313', '52318', '52319', '52322', '52327', '52330', '52331', '52339', '52343', '52344', '52354', '52355', '52356', '52357', '52358', '52359', '52360', '52364', '52365', '52366', '52367', '52368', '52369', '52370', '52371', '52475', '62373', '62385', '62390', '62394', '75854', '85935', '88122', '97302', '98246', '118424', '118425', '123856', '124861', '130219', '130220', '130221', '130755', '130858', '131102', '131109', '133559', '146553', '146554', '146555', '146710', '148954', '151565', '166454', '166457', '195162', '200006', '227398', '227400', '235294', '235296', '242238', '251317', '292016', '292020', '292021', '292023', '309598', '309600', '309605', '313901', '313903', '320928', '19594', '51490', '51491', '52410', '52411', '52420', '52421', '52426', '52427', '52428', '52430', '52433', '52436', '52437', '52441', '52447', '52458', '52461', '52466', '52467', '52468', '52470', '52471', '85939', '85945', '85947', '85950', '90643', '129617', '133965', '134092', '134136', '134137', '180452', '205404', '205405', '205929', '237203', '237836', '237837', '237839', '238111', '238119', '242204', '251320', '252821', '252822', '252823', '252824', '252827', '257455', '260523', '260524', '264247', '264249', '264250', '264251', '264252', '264253', '264255', '264256', '264257', '264258', '264512', '269927', '269928', '269929', '269930', '272224', '272225', '272226', '272227', '272228', '272229', '272230', '272231', '272232', '272233', '272235', '273868', '273869', '273870', '275516', '275517', '275519', '275521', '277650', '277651', '277652', '277653', '277654', '277655', '277656', '277657', '277773', '277844', '277845', '278054', '278178', '278179', '279002', '279970', '279971', '279972', '279973', '279974', '279975', '280211', '280213', '280214', '280215', '280217', '280218', '285701', '285717', '285728', '286313', '286322', '286326', '286327', '286328', '286330', '286342', '286343', '289051', '290610', '296398', '296583', '296585', '296993', '297004', '297639', '298400', '298401', '305708', '305709', '305776', '305800', '309503', '321131', '322936', '324354', '324356', '328291', '330321', '330359', '9516', '48133', '48134', '48135', '48181', '48520', '58773', '114353', '117629', '117630', '130108', '130553', '248036', '249651', '249654', '250366', '250803', '251161', '251162', '251310', '251311', '251925', '252919', '252920', '253218', '253835', '253964', '254718', '254759', '254760', '254823', '254841', '254889', '255542', '256773', '258292', '258613', '258842', '259580', '260502', '260503', '264233', '264915', '268253', '268497', '270853', '276645', '276646', '284373', '289265', '289266', '289268', '289269', '289272', '289366', '289367', '289369', '289494', '291399', '296776', '296812', '15774', '15775', '48111', '48112', '48118', '48154', '48232', '48233', '48234', '48257', '48262', '48272', '48312', '48313', '48391', '48393', '48396', '48400', '48405', '48584', '48694', '48700', '48762', '48765', '48776', '48778', '48779', '48783', '48784', '48798', '48799', '50199', '71039', '71040', '71042', '88108', '90481', '144152', '144153', '144154', '144156', '144158', '144159', '146294', '146455', '160603', '163532', '167775', '180636', '193454', '193455', '193457', '193467', '193468', '204916', '204917', '204919', '205052', '205053', '212382', '227201', '240212', '252411', '252808', '256221', '256222', '256289', '256294', '258777', '263099', '263101', '263159', '263160', '263161', '263162', '263164', '263165', '263166', '263167', '263168', '263775', '263776', '267399', '267403', '267404', '267536', '267787', '267798', '267799', '267935', '267936', '268064', '268065', '273398', '273399', '273400', '273553', '273555', '273556', '274848', '275848', '275893', '275895', '275897', '275900', '275902', '277625', '278194', '278195', '278553', '279673', '280391', '280392', '280393', '280394', '280395', '287212', '307530', '307533', '314156', '314158', '314160', '314163', '48057', '48108', '48152', '48161', '48235', '48269', '48276', '48277', '48281', '48285', '48293', '48314', '48384', '48398', '48447', '48449', '48450', '48451', '48452', '48453', '48454', '48455', '48458', '48460', '48461', '48463', '48780', '48782', '85790', '137571', '138298', '157706', '164566', '178606', '180638', '180639', '182857', '186229', '204920', '207886', '210642', '210643', '210644', '210645', '240215', '240219', '243231', '243232', '253738', '267401', '273401', '273402', '275847', '275849', '275868', '275891', '275892', '278193', '282578', '282579', '290673', '306485', '307531', '33350', '35256', '35937', '40564', '48301', '48539', '48542', '48546', '48562', '48570', '48580', '48582', '49662', '55423', '68160', '119377', '125016', '211670', '226118', '226127', '240189', '250584', '250585', '250586', '255649', '267349', '269920', '279713', '309603', '317548', '319556', '57993', '58028', '58030', '58051', '58055', '58058', '58082', '58083', '132556', '184252', '194314', '234748', '278180', '278181', '278182', '286323', '286324', '286325', '297724', '297739', '300826', '320117', '320167', '48480', '48483', '60671', '60678', '60679', '60694', '60699', '60700', '60702', '60703', '60715', '60721', '60722', '60727', '60732', '60741', '170316', '170317', '170542', '187058', '247678', '258385', '258386', '258387', '258388', '258389', '258390', '258391', '258392', '258393', '258394', '258395', '258396', '258787', '258788', '258789', '258790', '258791', '258793', '258794', '258795', '258796', '258797', '258798', '258799', '261357', '263117', '263118', '263119', '263120', '263121', '263122', '263123', '263125', '263126', '263128', '263129', '263130', '263132', '263133', '263134', '263135', '263136', '263138', '263139', '263140', '263143', '263144', '263145', '263146', '263147', '263148', '263149', '263151', '263152', '263153', '263154', '288181', '313856', '19121', '34438', '50961', '61309', '106569', '106570', '106571', '109313', '109315', '109316', '122921', '122922', '122923', '123328', '123329', '123330', '130744', '130745', '130746', '147769', '147770', '147771', '174838', '174839', '174840', '175445', '175446', '175447', '193477', '193483', '193486', '198685', '198686', '198687', '206039', '206040', '206041', '213130', '213131', '213132', '219754', '219755', '226772', '226773', '226774', '227518', '234692', '252940', '271342', '272551', '299184', '299186', '48219', '66112', '66190', '66191', '66192', '117457', '186440', '273155', '273157', '273158', '277252', '279265', '279266', '68994', '69024', '69025', '69089', '69093', '69094', '69096', '69097', '69128', '69139', '69140', '69141', '69142', '69150', '69151', '69158', '69161', '69162', '69164', '69166', '69170', '69171', '69172', '69175', '69176', '69177', '69178', '69179', '69254', '69260', '69278', '69283', '69284', '69289', '69290', '69291', '69298', '69301', '69305', '69307', '69308', '69309', '69310', '69312', '69316', '69317', '69321', '69323', '69347', '69351', '69352', '69353', '69354', '83717', '83719', '83721', '83723', '88136', '120625', '129551', '130106', '130107', '130736', '132709', '135269', '135270', '135271', '151239', '151240', '167187', '167190', '167288', '167289', '167291', '170040', '170065', '175528', '175891', '178306', '180792', '184484', '184485', '184486', '184487', '187458', '194286', '194312', '194313', '199262', '199282', '200220', '202397', '210767', '210770', '226176', '226712', '229316', '229317', '234412', '234733', '235126', '235127', '237204', '237209', '238897', '247307', '249617', '249659', '259880', '259881', '259884', '263993', '264012', '264016', '264501', '264504', '264505', '264507', '264510', '264511', '267302', '267303', '267304', '269931', '269932', '269933', '270160', '270695', '272234', '272236', '273865', '273866', '273867', '282539', '282558', '282559', '282561', '282562', '283979', '284337', '284353', '284358', '286117', '286433', '286434', '288135', '288137', '288140', '290284', '290398', '290399', '290435', '290443', '290500', '291208', '291334', '291335', '291336', '292229', '293301', '297028', '297037', '297042', '297052', '297064', '298273', '300100', '308705', '308715', '313391', '313893', '313895', '313896', '313897', '313899', '313900', '319712', '319713', '319714', '319809', '320931', '320933', '321227', '324226', '8947', '8952', '8956', '8964', '9711', '9712', '9713', '9714', '9717', '9722', '35608', '73022', '73144', '73145', '73146', '73147', '73155', '73166', '73207', '73208', '73216', '73218', '73227', '73234', '83885', '87210', '99569', '99570', '99572', '99678', '101019', '101021', '101022', '117800', '123050', '123051', '123052', '151648', '151649', '151650', '151651', '151652', '151732', '151809', '151810', '151811', '151907', '151912', '154029', '166991', '175660', '182929', '204320', '204321', '210648', '210655', '214175', '220323', '226173', '237960', '248276', '248277', '248279', '248281', '248282', '248283', '248290', '251371', '252538', '258599', '258601', '258602', '258606', '258607', '258608', '258609', '258611', '260138', '264423', '264424', '265222', '265223', '265304', '265307', '265312', '265415', '265416', '265417', '265418', '265419', '265420', '272239', '272241', '272242', '272243', '272244', '272245', '272246', '272255', '275001', '279318', '279319', '281978', '283630', '283631', '283632', '283633', '283634', '283635', '292121', '308843', '309544', '309545', '309546', '309547', '309548', '309550', '309551', '309552', '309553', '309554', '309555', '309556', '309577', '309578', '309579', '309580', '309581', '309582', '309583', '309586', '309587', '309588', '309589', '313858', '313860', '8955', '8960', '8963', '73148', '73158', '73181', '73184', '73197', '73209', '73213', '73229', '73250', '73251', '87209', '88140', '90035', '99571', '117391', '118085', '129748', '130218', '148773', '169712', '214177', '214333', '220313', '220320', '220325', '220328', '220437', '258605', '258610', '264422', '11783', '11784', '11787', '84358', '117522', '117523', '190224', '256628', '256629', '264464', '271643', '307882']

df4 = pd.DataFrame()

for i in range(0, len(a), 40):
    
    conn = psycopg2.connect(dbname='adb', host='172.16.10.22', port='5432',
                            user='rpa_robot', password='Qaz123123+')

    cur = conn.cursor(name='1583_first_part1')

    start_date = '2023-01-01'
    end_date = '2023-03-31'
    store_id = '985'

    query = f"""
        select source_product_id, sum(quantity) FROM "dwh_data"."qs_fact_sales" q where source_product_id in ({str(a[i:i+40])[1:-1]}) and source_store_id = 116 and date(order_date) between to_date('{start_date}', 'YYYY-MM-DD') and to_date('{end_date}', 'YYYY-MM-DD')
        group by source_product_id;
    """

    cur.execute(query)

    print('Executed')

    df5 = pd.DataFrame(cur.fetchall())

    df4 = pd.concat([df4, df5])

    cur.close()
    conn.close()

Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed
Executed


In [154]:
df4.columns = ['code', 'quantity']
df4 = df4.reset_index()
df4 = df4.drop(['index'], axis=1)
df4

,code,quantity
0,106572,884.877
1,34431,134.468
2,106567,274.527
3,106568,1460.960
4,106565,1069.594
...,...,...
467,99571,740.000
468,169712,943.000
469,118085,304.000
470,11784,6.584


In [156]:
for i in range(len(df4)):
    print(df4.loc[i, 'code'], df4.loc[i, 'quantity'])

106572 884.877
34431 134.468
106567 274.527
106568 1460.960
106565 1069.594
106564 1089.133
106699 2.914
34426 1.344
34441 15.092
106566 105.343
106563 1616.791
147765 632.227
147761 798.063
147764 755.140
147767 921.435
147763 694.231
147768 82.721
203379 285.340
205522 147.172
267398 48.668
58693 52.148
58718 0.392
15832 0.906
10732 2.514
30968 2.930
41233 12.258
58842 42.612
53906 0.238
58918 20.988
68704 1.374
53093 30.970
58928 13.650
41235 0.828
69282 6.106
58829 9.240
58727 0.938
212419 3.000
124383 7.716
116694 85.178
174784 2.948
191488 19.000
284822 8.000
9836 38.074
34545 408.274
32800 0.796
10728 3.010
34425 11.810
30937 1.776
10729 1.624
53095 56.318
47124 0.344
15791 14.882
34596 2.248
58039 6.658
30999 0.654
71328 11.930
58775 22.294
58099 1.794
71312 0.756
58625 25.694
119370 9.644
58098 1.956
73615 6.076
75937 11.622
70224 0.682
151680 1.306
169072 16.354
187461 10.206
190246 1.000
191487 11.000
151681 0.194
233832 8.794
237178 0.558
233360 9.856
233833 3.048
210986 2.

In [158]:
df_new[df_new['code'] == '99571']

,code,quantity,group


In [170]:
df4[df4['code'] == '99571']

,code,quantity
467,99571,740.000


In [188]:
df4['group'] = ''
for i in range(len(df4)):
    df4.loc[i, 'group'] = df_wnodes[df_wnodes['code'] == df4.loc[i, 'code']]['group'].iloc[0]

In [189]:
df4

,code,quantity,group
0,106572,884.877,"Kуры(включая цыплят), свежий и охл., части ту..."
1,34431,134.468,"Kуры(включая цыплят), свежий и охл., части ту..."
2,106567,274.527,"Kуры(включая цыплят), свежий и охл., части ту..."
3,106568,1460.960,"Kуры(включая цыплят), свежий и охл., части ту..."
4,106565,1069.594,"Kуры(включая цыплят), свежий и охл., части ту..."
...,...,...,...
467,99571,740.000,Хлеб ржаной
468,169712,943.000,Хлеб ржаной
469,118085,304.000,Хлеб ржаной
470,11784,6.584,Вареники


In [218]:
df_new

,code,quantity,name,group
0,52447,6.340,"ПИРОЖНОЕ НАПОЛЕОН, КГ",Пирожные свежие
1,58656,4.444,САЛАТ ЗАКУСКА РУССКАЯ,Блюда готовые рыбные
2,263051,59.592,ГОВЯДИНА ВЫРЕЗКА ОХЛ. КГ ЗАМОРОЗКА,Говядина свеж. или охлажд.: отруба
3,23492,1.210,КАРТОФЕЛЬ ПО-ДЕРЕВЕНСКИ,Блюда готовые овощные не замороженные
4,23307,7.416,КАРАСЬ ЖАРЕНЫЙ,Блюда готовые рыбные
...,...,...,...,...
467,169712,943.000,ХЛЕБ СЕРЫЙ ФОРМОВОЙ 400 Г,Хлеб ржаной
468,48580,2.008,П/Ф ТЕСТО СДОБНОЕ ДРОЖЖЕВОЕ,"Продукты пищ. из муки, крупы, крахмала, прочие"
469,125016,33.784,ЛАПША ДЛЯ ЛАГМАНА ОТВАРНАЯ ЗАМОРОЗКА,"Продукты пищ. из муки, крупы, крахмала, прочие"
470,106699,2.914,ТУШКА Ц/Б КУС ВКУС Д/ГРИЛЯ ОХЛ КГ ЛОТОК,"Kуры(включая цыплят), свежий и охл., части ту..."
